# Capstone Project - The Battle of the Neighborhoods

## Applied Data Science Capstone by IBM/Coursera

### Table of contents
Introduction: Business Problem
Data
Methodology
Analysis
Results and Discussion
Conclusion

#### Introduction: Business Problem 
This project aims to select the safest borough in London based on the total crimes, explore the neighborhoods of that borough to find the 10 most common venues in each neighborhood and finally cluster the neighborhoods using k-mean clustering.

This report will be targeted to people who are looking to relocate to London. Inorder to finalise a neighborhood to hunt for an apartment, safety is considered as a top concern when moving to a new place. If you don’t feel safe in your own home, you’re not going to be able to enjoy living there. The crime statistics will provide an insight into this issue.

We will focus on the safest borough and explore its neighborhoods and the 10 most common venues in each neighborhood so that the best neighborhood suited to an individual's needs can be selected.

### Data 
Based on definition of our problem, factors that will influence our decision are:

The total number of crimes commited in each of the borough during the last year.
The most common venues in each of the neighborhood in the safest borough selected.
Following data sources will be needed to extract/generate the required information:

Part 1: Preprocessing a real world data set from Kaggle showing the London Crimes from 2008 to 2016: A dataset consisting of the crime statistics of each borough in London obtained from Kaggle
Part 2: Scraping additional information of the different Boroughs in London from a Wikipedia page.: More information regarding the boroughs of London is scraped using the Beautifulsoup library
Part 3: Creating a new dataset of the Neighborhoods of the safest borough in London and generating their co-ordinates.: Co-ordinate of neighborhood will be obtained using Google Maps API geocoding

## Part 1: Preprocessing a real world data set from Kaggle showing the London Crimes from 2008 to 2016

### London Crime Data
About this file

lsoa_code: code for Lower Super Output Area in Greater London.
borough: Common name for London borough.
major_category: High level categorization of crime
minor_category: Low level categorization of crime within major category.
value: monthly reported count of categorical crime in given borough
year: Year of reported counts, 2008-2016
month: Month of reported counts, 1-12

Data set URL: https://www.kaggle.com/jboysen/london-crime

###### Import necessary libraries

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup # library for web scrapping  

!conda install -c conda-forge geocoder --yes
import geocoder

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    python_abi:    

ModuleNotFoundError: No module named 'folium'

#### Define Foursquare Credentials and Version

Make sure that you have created a Foursquare developer account and have your credentials handy

In [3]:
CLIENT_ID = 'R01LINGO2WC45KLRLKT3ZHU2QENAO2IPRK2N2ELOHRNK4P3K' # your Foursquare ID
CLIENT_SECRET = '4JT1TWRMXMPLX5IOKNBAFU3L3ARXK4D5JJDPFK1CLRZM2ZVW' # your Foursquare Secret

VERSION = '20180604'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: R01LINGO2WC45KLRLKT3ZHU2QENAO2IPRK2N2ELOHRNK4P3K
CLIENT_SECRET:4JT1TWRMXMPLX5IOKNBAFU3L3ARXK4D5JJDPFK1CLRZM2ZVW


Your credentails:
CLIENT_ID: R01LINGO2WC45KLRLKT3ZHU2QENAO2IPRK2N2ELOHRNK4P3K
CLIENT_SECRET:4JT1TWRMXMPLX5IOKNBAFU3L3ARXK4D5JJDPFK1CLRZM2ZVW

#### Read in the dataset

# Read in the data 
df = pd.read_csv("london_crime_by_lsoa.csv")

In [ ]:
# View the top rows of the dataset
df.head()

In [ ]:
index  isoa_code    borough    major_category   minor_category       year    month

0       E989gf9     Croydon     Burglary        Burglary 
                                                in Other Buildings	 2016	   11

1       E59gf9     Greenwich   Violence Against 
                                the Person	     Other violence    	 2016	   11

2       E59io9       Bromley    Violence Against 
                                the Person	     Other violence      2015	    5

3       E59kf9      Redbridge   Burglary	       Burglary in 
                                                Other Buildings      2016	    3

4       E59af9      Wandsworth  Robbery	         Personal Property    2008  	6

#### Accessing the most recent crime rates (2016)

In [ ]:
# Taking only the most recent year (2016) and dropping the rest
df.drop(df.index[df['year'] != 2016], inplace = True)

# Removing all the entires where crime values are null  
df = df[df.value != 0]

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

In [ ]:
# Shape of the data frame
df.shape

(392042, 7)

#### Change the column names

In [ ]:
df.columns = ['LSOA_Code', 'Borough','Major_Category','Minor_Category','No_of_Crimes','Year','Month']
df.head()

index  isoa_code    borough    major_category   minor_category       year    month

0       E989gf9     Croydon     Burglary        Burglary 
                                                in Other Buildings	 2016	   11

1       E59gf9     Greenwich   Violence Against 
                                the Person	     Other violence    	 2016	   11

2       E59io9       Bromley    Violence Against 
                                the Person	     Other violence      2015	    5

3       E59kf9      Redbridge   Burglary	       Burglary in 
                                                Other Buildings      2016	    3

4       E59af9      Wandsworth  Robbery	         Personal Property    2008  	6

In [ ]:
# View the information of the dataset 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392042 entries, 0 to 392041
Data columns (total 7 columns):
LSOA_Code         392042 non-null object
Borough           392042 non-null object
Major_Category    392042 non-null object
Minor_Category    392042 non-null object
No_of_Crimes      392042 non-null int64
Year              392042 non-null int64
Month             392042 non-null int64
dtypes: int64(3), object(4)
memory usage: 20.9+ MB

### Total number of crimes in each Borough

In [ ]:
df['Borough'].value_counts()

Lambeth                   17605
Southwark                 16560
Croydon                   16254
Newham                    15622
Ealing                    15284
Tower Hamlets             15219
Brent                     14980
Barnet                    14668
Hackney                   14392
Lewisham                  14235
Haringey                  14202
Enfield                   13658
Wandsworth                13498
Westminster               13383
Islington                 13116
Greenwich                 12750
Camden                    12632
Hillingdon                12417
Hounslow                  12316
Waltham Forest            12121
Bromley                   11980
Redbridge                 11490
Hammersmith and Fulham    10281
Barking and Dagenham       9784
Havering                   9699
Kensington and Chelsea     9653
Harrow                     8257
Bexley                     8245
Merton                     8223
Richmond upon Thames       7199
Sutton                     6823
Kingston upon Thames       5374
City of London              122
Name: Borough, dtype: int64

##### The total crimes per major category

In [ ]:
df['Major_Category'].value_counts()

Theft and Handling             129159
Violence Against the Person    123050
Criminal Damage                 48584
Burglary                        43020
Drugs                           21782
Robbery                         14889
Other Notifiable Offences       11558
Name: Major_Category, dtype: int64

### Pivoting the table to view the no. of crimes for each major category in each Borough

In [ ]:
London_crime = pd.pivot_table(df,values=['No_of_Crimes'],
                               index=['Borough'],
                               columns=['Major_Category'],
                               aggfunc=np.sum,fill_value=0)
London_crime.head()

index  isoa_code    borough    major_category   minor_category       year    month

0       E989gf9     Croydon     Burglary        Burglary 
                                                in Other Buildings	 2016	   11

1       E59gf9     Greenwich   Violence Against 
                                the Person	     Other violence    	 2016	   11

2       E59io9       Bromley    Violence Against 
                                the Person	     Other violence      2015	    5

3       E59kf9      Redbridge   Burglary	       Burglary in 
                                                Other Buildings      2016	    3

4       E59af9      Wandsworth  Robbery	         Personal Property    2008  	6

In [ ]:
# Reset the index
London_crime.reset_index(inplace = True)

In [ ]:
# Total crimes per Borough
London_crime['Total'] = London_crime.sum(axis=1)
London_crime.head(33)

index  isoa_code    borough    major_category   minor_category       year    month

0       E989gf9     Croydon     Burglary        Burglary 
                                                in Other Buildings	 2016	   11

1       E59gf9     Greenwich   Violence Against 
                                the Person	     Other violence    	 2016	   11

2       E59io9       Bromley    Violence Against 
                                the Person	     Other violence      2015	    5

3       E59kf9      Redbridge   Burglary	       Burglary in 
                                                Other Buildings      2016	    3

4       E59af9      Wandsworth  Robbery	         Personal Property    2008  	6

5       E989gf9     Croydon     Burglary        Burglary 
                                                in Other Buildings	 2016	   11

6       E59gf9     Greenwich   Violence Against 
                                the Person	     Other violence    	 2016	   11

8       E59io9       Bromley    Violence Against 
                                the Person	     Other violence      2015	    5

9       E59kf9      Redbridge   Burglary	       Burglary in 
                                                Other Buildings      2016	    3

10       E59af9      Wandsworth  Robbery	         Personal Property    2008  	

11       E989gf9     Croydon     Burglary        Burglary 
                                                in Other Buildings	 2016	   11

12       E59gf9     Greenwich   Violence Against 
                                the Person	     Other violence    	 2016	   11

13       E59io9       Bromley    Violence Against 
                                the Person	     Other violence      2015	    5

14       E59kf9      Redbridge   Burglary	       Burglary in 
                                                Other Buildings      2016	    3

15       E59af9      Wandsworth  Robbery	         Personal Property    2008  	

16       E989gf9     Croydon     Burglary        Burglary 
                                                in Other Buildings	 2016	   11

17       E59gf9     Greenwich   Violence Against 
                                the Person	     Other violence    	 2016	   11

18       E59io9       Bromley    Violence Against 
                                the Person	     Other violence      2015	    5

19       E59kf9      Redbridge   Burglary	       Burglary in 
                                                Other Buildings      2016	    3

20      E59af9      Wandsworth  Robbery	         Personal Property    2008  	6


21       E989gf9     Croydon     Burglary        Burglary 
                                                in Other Buildings	 2016	   11

22       E59gf9     Greenwich   Violence Against 
                                the Person	     Other violence    	 2016	   11

23       E59io9       Bromley    Violence Against 
                                the Person	     Other violence      2015	    5

24       E59kf9      Redbridge   Burglary	       Burglary in 
                                                Other Buildings      2016	    3

25       E59af9      Wandsworth  Robbery	         Personal Property    2008  	

#### Removing the multi index so that it will be easier to merge

In [ ]:
London_crime.columns = London_crime.columns.map(''.join)
London_crime.head()

index  isoa_code    borough    major_category   minor_category       year    month

0       E989gf9     Croydon     Burglary        Burglary 
                                                in Other Buildings	 2016	   11

1       E59gf9     Greenwich   Violence Against 
                                the Person	     Other violence    	 2016	   11

2       E59io9       Bromley    Violence Against 
                                the Person	     Other violence      2015	    5

3       E59kf9      Redbridge   Burglary	       Burglary in 
                                                Other Buildings      2016	    3

4       E59af9      Wandsworth  Robbery	         Personal Property    2008  	6

#### Renaming the columns

In [ ]:
London_crime.columns = ['Borough','Burglary', 'Criminal Damage','Drugs','Other Notifiable Offences',
                        'Robbery','Theft and Handling','Violence Against the Person','Total']
London_crime.head()

index  isoa_code    borough    major_category   minor_category       year    month

0       E989gf9     Croydon     Burglary        Burglary 
                                                in Other Buildings	 2016	   11

1       E59gf9     Greenwich   Violence Against 
                                the Person	     Other violence    	 2016	   11

2       E59io9       Bromley    Violence Against 
                                the Person	     Other violence      2015	    5

3       E59kf9      Redbridge   Burglary	       Burglary in 
                                                Other Buildings      2016	    3

4       E59af9      Wandsworth  Robbery	         Personal Property    2008  	6

In [ ]:
# Shape of the data set 
London_crime.shape

(33, 9)

In [11]:
# View the Columns in the data frame
# London_crime.columns.tolist()

## Part 2: Scraping additional information of the different Boroughs in London from a Wikipedia page 
Using Beautiful soup to scrap the latitude and longitiude of the boroughs in London

URL: https://en.wikipedia.org/wiki/List_of_London_boroughs

In [ ]:
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_London_boroughs'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="UTF-8"/>
  <title>
   List of London boroughs - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_London_boroughs","wgTitle":"List of London boroughs","wgCurRevisionId":881899861,"wgRevisionId":881899861,"wgArticleId":28092685,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from August 2015","Use British English from August 2015","Lists of coordinates","Geographic coordinate lists","Articles with Geo","London boroughs","Lists of places in London"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRelevantPageName":"List_of_London_boroughs","wgRelevantArticleId":28092685,"wgRequestId":"XMbUCwpAAEUAACKj1V4AAABQ","wgCSPNonce":false,"wgIsProbablyEditable":true,"wgRelevantPageIsProbablyEditable":true,"wgRestrictionEdit":[],"wgRestrictionMove":[],"wgFlaggedRevsParams":{"tags":{}},"wgStableRevisionId":null,"wgMediaViewerOnClick":true,"wgMediaViewerEnabledByDefault":true,"wgPopupsReferencePreviews":false,"wgPopupsConflictsWithNavPopupGadget":false,"wgVisualEditor":{"pageLanguageCode":"en","pageLanguageDir":"ltr","pageVariantFallbacks":"en","usePageImages":true,"usePageDescriptions":true},"wgMFDisplayWikibaseDescriptions":{"search":true,"nearby":true,"watchlist":true,"tagline":false},"wgRelatedArticles":null,"wgRelatedArticlesUseCirrusSearch":true,"wgRelatedArticlesOnlyUseCirrusSearch":false,"wgWMESchemaEditAttemptStepOversample":false,"wgPoweredByHHVM":true,"wgULSCurrentAutonym":"English","wgNoticeProject":"wikipedia","wgCentralNoticeCookiesToDelete":[],"wgCentralNoticeCategoriesUsingLegacy":["Fundraising","fundraising"],"wgWikibaseItemId":"Q6577004","wgCentralAuthMobileDomain":false,"wgEditSubmitButtonLabelPublish":true});mw.loader.state({"ext.gadget.charinsert-styles":"ready","ext.globalCssJs.user.styles":"ready","ext.globalCssJs.site.styles":"ready","site.styles":"ready","noscript":"ready","user.styles":"ready","ext.globalCssJs.user":"ready","ext.globalCssJs.site":"ready","user":"ready","user.options":"ready","user.tokens":"loading","ext.cite.styles":"ready","mediawiki.legacy.shared":"ready","mediawiki.legacy.commonPrint":"ready","mediawiki.toc.styles":"ready","wikibase.client.init":"ready","ext.visualEditor.desktopArticleTarget.noscript":"ready","ext.uls.interlanguage":"ready","ext.wikimediaBadges":"ready","ext.3d.styles":"ready","mediawiki.skinning.interface":"ready","skins.vector.styles":"ready"});mw.loader.implement("user.tokens@0tffind",function($,jQuery,require,module){/*@nomin*/mw.user.tokens.set({"editToken":"+\\","patrolToken":"+\\","watchToken":"+\\","csrfToken":"+\\"});
});RLPAGEMODULES=["ext.cite.ux-enhancements","site","mediawiki.page.startup","mediawiki.page.ready","jquery.tablesorter","mediawiki.toc","mediawiki.searchSuggest","ext.gadget.teahouse","ext.gadget.ReferenceTooltips","ext.gadget.watchlist-notice","ext.gadget.DRN-wizard","ext.gadget.charinsert","ext.gadget.refToolbar","ext.gadget.extra-toolbar-buttons","ext.gadget.switcher","ext.centralauth.centralautologin","mmv.head","mmv.bootstrap.autostart","ext.popups","ext.visualEditor.desktopArticleTarget.init","ext.visualEditor.targetLoader","ext.eventLogging","ext.wikimediaEvents","ext.navigationTiming","ext.uls.eventlogger","ext.uls.init","ext.uls.compactlinks","ext.uls.interface","ext.quicksurveys.init","ext.centralNotice.geoIP","ext.centralNotice.startUp","skins.vector.js"];mw.loader.load(RLPAGEMODULES);});
  </script>
  <link href="/w/load.php?lang=en&amp;modules=ext.3d.styles%7Cext.cite.styles%7Cext.uls.interlanguage%7Cext.visualEditor.desktopArticleTarget.noscript%7Cext.wikimediaBadges%7Cmediawiki.legacy.commonPrint%2Cshared%7Cmediawiki.skinning.interface%7Cmediawiki.toc.styles%7Cskins.vector.styles%7Cwikibase.client.init&amp;only=styles&amp;skin=vector" rel="stylesheet"/>
  <script async="" src="/w/load.php?lang=en&amp;modules=startup&amp;only=scripts&amp;skin=vector"/>
  <meta content="" name="ResourceLoaderDynamicStyles"/>
  <link href="/w/load.php?lang=en&amp;modules=ext.gadget.charinsert-styles&amp;only=styles&amp;skin=vector" rel="stylesheet"/>
  <link href="/w/load.php?lang=en&amp;modules=site.styles&amp;only=styles&amp;skin=vector" rel="stylesheet"/>
  <meta content="MediaWiki 1.34.0-wmf.1" name="generator"/>
  <meta content="origin" name="referrer"/>
  <meta content="origin-when-crossorigin" name="referrer"/>
  <meta content="origin-when-cross-origin" name="referrer"/>
  <link href="android-app://org.wikipedia/http/en.m.wikipedia.org/wiki/List_of_London_boroughs" rel="alternate"/>
  <link href="/w/index.php?title=List_of_London_boroughs&amp;action=edit" rel="alternate" title="Edit this page" type="application/x-wiki"/>
  <link href="/w/index.php?title=List_of_London_boroughs&amp;action=edit" rel="edit" title="Edit this page"/>
  <link href="/static/apple-touch/wikipedia.png" rel="apple-touch-icon"/>
  <link href="/static/favicon/wikipedia.ico" rel="shortcut icon"/>
  <link href="/w/opensearch_desc.php" rel="search" title="Wikipedia (en)" type="application/opensearchdescription+xml"/>
  <link href="//en.wikipedia.org/w/api.php?action=rsd" rel="EditURI" type="application/rsd+xml"/>
  <link href="//creativecommons.org/licenses/by-sa/3.0/" rel="license"/>
  <link href="https://en.wikipedia.org/wiki/List_of_London_boroughs" rel="canonical"/>
  <link href="//login.wikimedia.org" rel="dns-prefetch"/>
  <link href="//meta.wikimedia.org" rel="dns-prefetch"/>
  <!--[if lt IE 9]><script src="/w/load.php?lang=qqx&amp;modules=html5shiv&amp;only=scripts&amp;skin=fallback&amp;sync=1"></script><![endif]-->
 </head>
 <body class="mediawiki ltr sitedir-ltr mw-hide-empty-elt ns-0 ns-subject mw-editable page-List_of_London_boroughs rootpage-List_of_London_boroughs skin-vector action-view">
  <div class="noprint" id="mw-page-base"/>
  <div class="noprint" id="mw-head-base"/>
  <div class="mw-body" id="content" role="main">
   <a id="top"/>
   <div class="mw-body-content" id="siteNotice">
    <!-- CentralNotice -->
   </div>
   <div class="mw-indicators mw-body-content">
   </div>
   <h1 class="firstHeading" id="firstHeading" lang="en">
    List of London boroughs
   </h1>
   <div class="mw-body-content" id="bodyContent">
    <div class="noprint" id="siteSub">
     From Wikipedia, the free encyclopedia
    </div>
    <div id="contentSub"/>
    <div id="jump-to-nav"/>
    <a class="mw-jump-link" href="#mw-head">
     Jump to navigation
    </a>
    <a class="mw-jump-link" href="#p-search">
     Jump to search
    </a>
    <div class="mw-content-ltr" dir="ltr" id="mw-content-text" lang="en">
     <div class="mw-parser-output">
      <p class="mw-empty-elt">
      </p>
      <p class="mw-empty-elt">
      </p>
      <div class="thumb tright">
       <div class="thumbinner" style="width:302px;">
        <a class="image" href="/wiki/File:London-boroughs.svg">
         <img alt="" class="thumbimage" data-file-height="386" data-file-width="489" decoding="async" height="237" src="//upload.wikimedia.org/wikipedia/commons/thumb/2/29/London-boroughs.svg/300px-London-boroughs.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/2/29/London-boroughs.svg/450px-London-boroughs.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/2/29/London-boroughs.svg/600px-London-boroughs.svg.png 2x" width="300"/>
        </a>
        <div class="thumbcaption">
         <div class="magnify">
          <a class="internal" href="/wiki/File:London-boroughs.svg" title="Enlarge"/>
         </div>
         Map of the 32 London boroughs and the City of London.
        </div>
       </div>
      </div>
      <p>
       This is a list of
       <a href="/wiki/Districts_of_England" title="Districts of England">
        local authority districts
       </a>
       within
       <a href="/wiki/Greater_London" title="Greater London">
        Greater London
       </a>
       , including 32
       <a href="/wiki/London_boroughs" title="London boroughs">
        London boroughs
       </a>
       and the
       <a href="/wiki/City_of_London" title="City of London">
        City of London
       </a>
       . The London boroughs were all created on 1 April 1965. Upon creation, twelve were designated
       <a href="/wiki/Inner_London" title="Inner London">
        Inner London
       </a>
       boroughs and the remaining twenty were designated
       <a href="/wiki/Outer_London" title="Outer London">
        Outer London
       </a>
       boroughs. The
       <a href="/wiki/Office_for_National_Statistics" title="Office for National Statistics">
        Office for National Statistics
       </a>
       has amended the designations of three boroughs for statistics purposes only. Three boroughs have been granted the designation
       <a class="mw-redirect" href="/wiki/Royal_borough" title="Royal borough">
        royal borough
       </a>
       and one has
       <a href="/wiki/City_status_in_the_United_Kingdom" title="City status in the United Kingdom">
        city status
       </a>
       . For planning purposes, in addition to the boroughs and City there are also two active development corporations, the
       <a href="/wiki/London_Legacy_Development_Corporation" title="London Legacy Development Corporation">
        London Legacy Development Corporation
       </a>
       and
       <a href="/wiki/Old_Oak_and_Park_Royal_Development_Corporation" title="Old Oak and Park Royal Development Corporation">
        Old Oak and Park Royal Development Corporation
       </a>
       .
      </p>
      <div class="toc" id="toc">
       <input class="toctogglecheckbox" id="toctogglecheckbox" role="button" style="display:none" type="checkbox"/>
       <div class="toctitle" dir="ltr" lang="en">
        <h2>
         Contents
        </h2>
        <span class="toctogglespan">
         <label class="toctogglelabel" for="toctogglecheckbox"/>
        </span>
       </div>
       <ul>
        <li class="toclevel-1 tocsection-1">
         <a href="#List_of_boroughs_and_local_authorities">
          <span class="tocnumber">
           1
          </span>
          <span class="toctext">
           List of boroughs and local authorities
          </span>
         </a>
        </li>
        <li class="toclevel-1 tocsection-2">
         <a href="#City_of_London">
          <span class="tocnumber">
           2
          </span>
          <span class="toctext">
           City of London
          </span>
         </a>
        </li>
        <li class="toclevel-1 tocsection-3">
         <a href="#See_also">
          <span class="tocnumber">
           3
          </span>
          <span class="toctext">
           See also
          </span>
         </a>
        </li>
        <li class="toclevel-1 tocsection-4">
         <a href="#Notes">
          <span class="tocnumber">
           4
          </span>
          <span class="toctext">
           Notes
          </span>
         </a>
        </li>
        <li class="toclevel-1 tocsection-5">
         <a href="#References">
          <span class="tocnumber">
           5
          </span>
          <span class="toctext">
           References
          </span>
         </a>
        </li>
        <li class="toclevel-1 tocsection-6">
         <a href="#External_links">
          <span class="tocnumber">
           6
          </span>
          <span class="toctext">
           External links
          </span>
         </a>
        </li>
       </ul>
      </div>
      <h2>
       <span class="mw-headline" id="List_of_boroughs_and_local_authorities">
        List of boroughs and local authorities
       </span>
       <span class="mw-editsection">
        <span class="mw-editsection-bracket">
         [
        </span>
        <a href="/w/index.php?title=List_of_London_boroughs&amp;action=edit&amp;section=1" title="Edit section: List of boroughs and local authorities">
         edit
        </a>
        <span class="mw-editsection-bracket">
         ]
        </span>
       </span>
      </h2>
      <table class="wikitable sortable" style="font-size:100%" width="100%">
       <tbody>
        <tr>
         <th>
          Borough
         </th>
         <th>
          Inner
         </th>
         <th>
          Status
         </th>
         <th>
          Local authority
         </th>
         <th>
          Political control
         </th>
         <th>
          Headquarters
         </th>
         <th>
          Area (sq mi)
         </th>
         <th>
          Population (2013 est)
          <sup class="reference" id="cite_ref-1">
           <a href="#cite_note-1">
            [1]
           </a>
          </sup>
         </th>
         <th>
          Co-ordinates
         </th>
         <th>
          <span style="background:#67BCD3">
           Nr. in map
          </span>
         </th>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Barking_and_Dagenham" title="London Borough of Barking and Dagenham">
           Barking and Dagenham
          </a>
          <sup class="reference" id="cite_ref-2">
           <a href="#cite_note-2">
            [note 1]
           </a>
          </sup>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Barking_and_Dagenham_London_Borough_Council" title="Barking and Dagenham London Borough Council">
           Barking and Dagenham London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Barking_Town_Hall&amp;action=edit&amp;redlink=1" title="Barking Town Hall (page does not exist)">
           Town Hall
          </a>
          , 1 Town Square
         </td>
         <td>
          13.93
         </td>
         <td>
          194,352
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5607_N_0.1557_E_region:GB_type:city&amp;title=Barking+and+Dagenham">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°33′39″N
              </span>
              <span class="longitude">
               0°09′21″E
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5607°N 0.1557°E
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5607; 0.1557
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Barking and Dagenham
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          25
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Barnet" title="London Borough of Barnet">
           Barnet
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Barnet_London_Borough_Council" title="Barnet London Borough Council">
           Barnet London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">
           Conservative
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=North_London_Business_Park&amp;action=edit&amp;redlink=1" title="North London Business Park (page does not exist)">
           North London Business Park
          </a>
          , Oakleigh Road South
         </td>
         <td>
          33.49
         </td>
         <td>
          369,088
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.6252_N_0.1517_W_region:GB_type:city&amp;title=Barnet">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°37′31″N
              </span>
              <span class="longitude">
               0°09′06″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.6252°N 0.1517°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.6252; -0.1517
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Barnet
               </span>
                     )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          31
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Bexley" title="London Borough of Bexley">
           Bexley
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Bexley_London_Borough_Council" title="Bexley London Borough Council">
           Bexley London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">
           Conservative
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Civic_Offices&amp;action=edit&amp;redlink=1" title="Civic Offices (page does not exist)">
           Civic Offices
          </a>
          , 2 Watling Street
         </td>
         <td>
          23.38
         </td>
         <td>
          236,687
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4549_N_0.1505_E_region:GB_type:city&amp;title=Bexley">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°27′18″N
              </span>
              <span class="longitude">
               0°09′02″E
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.4549°N 0.1505°E
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.4549; 0.1505
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Bexley
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          23
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Brent" title="London Borough of Brent">
           Brent
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Brent_London_Borough_Council" title="Brent London Borough Council">
           Brent London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a href="/wiki/Brent_Civic_Centre" title="Brent Civic Centre">
           Brent Civic Centre
          </a>
          , Engineers Way
         </td>
         <td>
          16.70
         </td>
         <td>
          317,264
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5588_N_0.2817_W_region:GB_type:city&amp;title=Brent">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°33′32″N
              </span>
              <span class="longitude">
               0°16′54″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5588°N 0.2817°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5588; -0.2817
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Brent
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          12
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Bromley" title="London Borough of Bromley">
           Bromley
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Bromley_London_Borough_Council" title="Bromley London Borough Council">
           Bromley London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">
           Conservative
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Bromley_Civic_Centre&amp;action=edit&amp;redlink=1" title="Bromley Civic Centre (page does not exist)">
           Civic Centre
          </a>
          , Stockwell Close
         </td>
         <td>
          57.97
         </td>
         <td>
          317,899
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4039_N_0.0198_E_region:GB_type:city&amp;title=Bromley">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°24′14″N
              </span>
              <span class="longitude">
               0°01′11″E
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.4039°N 0.0198°E
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.4039; 0.0198
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Bromley
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          20
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Camden" title="London Borough of Camden">
           Camden
          </a>
         </td>
         <td>
          <img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/>
          <span style="display:none">
           Y
          </span>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Camden_London_Borough_Council" title="Camden London Borough Council">
           Camden London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a href="/wiki/Camden_Town_Hall" title="Camden Town Hall">
           Camden Town Hall
          </a>
          , Judd Street
         </td>
         <td>
          8.40
         </td>
         <td>
          229,719
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.529_N_0.1255_W_region:GB_type:city&amp;title=Camden">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°31′44″N
              </span>
              <span class="longitude">
               0°07′32″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5290°N 0.1255°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5290; -0.1255
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Camden
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          11
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Croydon" title="London Borough of Croydon">
           Croydon
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Croydon_London_Borough_Council" title="Croydon London Borough Council">
           Croydon London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Bernard_Weatherill_House&amp;action=edit&amp;redlink=1" title="Bernard Weatherill House (page does not exist)">
           Bernard Weatherill House
          </a>
          , Mint Walk
         </td>
         <td>
          33.41
         </td>
         <td>
          372,752
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.3714_N_0.0977_W_region:GB_type:city&amp;title=Croydon">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°22′17″N
              </span>
              <span class="longitude">
               0°05′52″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.3714°N 0.0977°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.3714; -0.0977
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Croydon
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          19
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Ealing" title="London Borough of Ealing">
           Ealing
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Ealing_London_Borough_Council" title="Ealing London Borough Council">
           Ealing London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Perceval_House,_Ealing&amp;action=edit&amp;redlink=1" title="Perceval House, Ealing (page does not exist)">
           Perceval House
          </a>
          , 14-16 Uxbridge Road
         </td>
         <td>
          21.44
         </td>
         <td>
          342,494
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.513_N_0.3089_W_region:GB_type:city&amp;title=Ealing">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°30′47″N
              </span>
              <span class="longitude">
               0°18′32″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5130°N 0.3089°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5130; -0.3089
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Ealing
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          13
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Enfield" title="London Borough of Enfield">
           Enfield
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Enfield_London_Borough_Council" title="Enfield London Borough Council">
           Enfield London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Enfield_Civic_Centre&amp;action=edit&amp;redlink=1" title="Enfield Civic Centre (page does not exist)">
           Civic Centre
          </a>
          , Silver Street
         </td>
         <td>
          31.74
            </td>
         <td>
          320,524
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.6538_N_0.0799_W_region:GB_type:city&amp;title=Enfield">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°39′14″N
              </span>
              <span class="longitude">
               0°04′48″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.6538°N 0.0799°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.6538; -0.0799
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Enfield
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          30
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/Royal_Borough_of_Greenwich" title="Royal Borough of Greenwich">
           Greenwich
          </a>
          <sup class="reference" id="cite_ref-3">
           <a href="#cite_note-3">
            [note 2]
           </a>
          </sup>
         </td>
         <td>
          <img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/>
          <span style="display:none">
           Y
          </span>
          <sup class="reference" id="cite_ref-note2_4-0">
           <a href="#cite_note-note2-4">
            [note 3]
           </a>
          </sup>
         </td>
         <td>
          <a class="mw-redirect" href="/wiki/Royal_borough" title="Royal borough">
           Royal
          </a>
         </td>
         <td>
          <a href="/wiki/Greenwich_London_Borough_Council" title="Greenwich London Borough Council">
           Greenwich London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a href="/wiki/Woolwich_Town_Hall" title="Woolwich Town Hall">
           Woolwich Town Hall
          </a>
          , Wellington Street
         </td>
         <td>
          18.28
         </td>
         <td>
          264,008
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4892_N_0.0648_E_region:GB_type:city&amp;title=Greenwich">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°29′21″N
              </span>
              <span class="longitude">
               0°03′53″E
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.4892°N 0.0648°E
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.4892; 0.0648
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Greenwich
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          22
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Hackney" title="London Borough of Hackney">
           Hackney
          </a>
         </td>
         <td>
          <img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/>
          <span style="display:none">
           Y
          </span>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Hackney_London_Borough_Council" title="Hackney London Borough Council">
           Hackney London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Hackney_Town_Hall&amp;action=edit&amp;redlink=1" title="Hackney Town Hall (page does not exist)">
           Hackney Town Hall
          </a>
          , Mare Street
         </td>
         <td>
          7.36
         </td>
         <td>
          257,379
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.545_N_0.0553_W_region:GB_type:city&amp;title=Hackney">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°32′42″N
              </span>
              <span class="longitude">
               0°03′19″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5450°N 0.0553°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5450; -0.0553
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Hackney
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          9
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Hammersmith_and_Fulham" title="London Borough of Hammersmith and Fulham">
           Hammersmith and Fulham
          </a>
          <sup class="reference" id="cite_ref-5">
           <a href="#cite_note-5">
            [note 4]
           </a>
          </sup>
         </td>
         <td>
          <img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/>
          <span style="display:none">
           Y
          </span>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Hammersmith_and_Fulham_London_Borough_Council" title="Hammersmith and Fulham London Borough Council">
           Hammersmith and Fulham London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Hammersmith_and_Fulham_Town_Hall&amp;action=edit&amp;redlink=1" title="Hammersmith and Fulham Town Hall (page does not exist)">
           Town Hall
          </a>
          , King Street
         </td>
         <td>
          6.33
         </td>
         <td>
          178,685
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4927_N_0.2339_W_region:GB_type:city&amp;title=Hammersmith+and+Fulham">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°29′34″N
              </span>
              <span class="longitude">
               0°14′02″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.4927°N 0.2339°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.4927; -0.2339
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Hammersmith and Fulham
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          4
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Haringey" title="London Borough of Haringey">
           Haringey
          </a>
         </td>
         <td>
          <sup class="reference" id="cite_ref-note2_4-1">
           <a href="#cite_note-note2-4">
            [note 3]
           </a>
          </sup>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Haringey_London_Borough_Council" title="Haringey London Borough Council">
           Haringey London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Haringey_Civic_Centre&amp;action=edit&amp;redlink=1" title="Haringey Civic Centre (page does not exist)">
           Civic Centre
          </a>
          , High Road
         </td>
         <td>
          11.42
         </td>
         <td>
          263,386
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.6_N_0.1119_W_region:GB_type:city&amp;title=Haringey">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°36′00″N
              </span>
              <span class="longitude">
               0°06′43″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.6000°N 0.1119°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.6000; -0.1119
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Haringey
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          29
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Harrow" title="London Borough of Harrow">
           Harrow
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Harrow_London_Borough_Council" title="Harrow London Borough Council">
           Harrow London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Harrow_Civic_Centre&amp;action=edit&amp;redlink=1" title="Harrow Civic Centre (page does not exist)">
           Civic Centre
          </a>
          , Station Road
         </td>
         <td>
          19.49
         </td>
         <td>
          243,372
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5898_N_0.3346_W_region:GB_type:city&amp;title=Harrow">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°35′23″N
              </span>
              <span class="longitude">
               0°20′05″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5898°N 0.3346°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5898; -0.3346
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Harrow
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          32
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Havering" title="London Borough of Havering">
           Havering
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Havering_London_Borough_Council" title="Havering London Borough Council">
           Havering London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">
           Conservative
          </a>
          (council
          <a href="/wiki/No_overall_control" title="No overall control">
           NOC
          </a>
          )
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Havering_Town_Hall&amp;action=edit&amp;redlink=1" title="Havering Town Hall (page does not exist)">
           Town Hall
          </a>
          , Main Road
         </td>
         <td>
          43.35
         </td>
         <td>
          242,080
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5812_N_0.1837_E_region:GB_type:city&amp;title=Havering">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°34′52″N
              </span>
              <span class="longitude">
               0°11′01″E
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5812°N 0.1837°E
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5812; 0.1837
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Havering
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          24
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Hillingdon" title="London Borough of Hillingdon">
           Hillingdon
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Hillingdon_London_Borough_Council" title="Hillingdon London Borough Council">
           Hillingdon London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">
           Conservative
          </a>
         </td>
         <td>
          <a href="/wiki/Hillingdon_Civic_Centre" title="Hillingdon Civic Centre">
           Civic Centre
          </a>
          , High Street
         </td>
         <td>
          44.67
         </td>
         <td>
          286,806
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5441_N_0.476_W_region:GB_type:city&amp;title=Hillingdon">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°32′39″N
              </span>
              <span class="longitude">
               0°28′34″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5441°N 0.4760°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5441; -0.4760
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Hillingdon
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          33
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Hounslow" title="London Borough of Hounslow">
           Hounslow
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Hounslow_London_Borough_Council" title="Hounslow London Borough Council">
           Hounslow London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Hounslow_Civic_Centre&amp;action=edit&amp;redlink=1" title="Hounslow Civic Centre (page does not exist)">
           Civic Centre
          </a>
          , Lampton Road
         </td>
         <td>
          21.61
         </td>
         <td>
          262,407
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4746_N_0.368_W_region:GB_type:city&amp;title=Hounslow">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°28′29″N
              </span>
              <span class="longitude">
               0°22′05″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.4746°N 0.3680°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.4746; -0.3680
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Hounslow
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          14
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Islington" title="London Borough of Islington">
           Islington
          </a>
         </td>
         <td>
          <img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/>
          <span style="display:none">
           Y
          </span>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Islington_London_Borough_Council" title="Islington London Borough Council">
           Islington London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Islington_Municipal_Offices&amp;action=edit&amp;redlink=1" title="Islington Municipal Offices (page does not exist)">
           Municipal Offices
          </a>
          , 222 Upper Street
         </td>
         <td>
          5.74
         </td>
         <td>
          215,667
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5416_N_0.1022_W_region:GB_type:city&amp;title=Islington">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°32′30″N
              </span>
              <span class="longitude">
               0°06′08″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5416°N 0.1022°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5416; -0.1022
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Islington
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          10
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/Royal_Borough_of_Kensington_and_Chelsea" title="Royal Borough of Kensington and Chelsea">
           Kensington and Chelsea
          </a>
         </td>
         <td>
          <img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/>
          <span style="display:none">
           Y
          </span>
         </td>
         <td>
          <a class="mw-redirect" href="/wiki/Royal_borough" title="Royal borough">
           Royal
          </a>
         </td>
         <td>
          <a href="/wiki/Kensington_and_Chelsea_London_Borough_Council" title="Kensington and Chelsea London Borough Council">
           Kensington and Chelsea London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">
           Conservative
          </a>
         </td>
         <td>
          <a href="/wiki/Kensington_Town_Hall,_London" title="Kensington Town Hall, London">
           The Town Hall
          </a>
          ,
          <a href="/wiki/Hornton_Street" title="Hornton Street">
           Hornton Street
          </a>
         </td>
         <td>
          4.68
         </td>
         <td>
          155,594
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.502_N_0.1947_W_region:GB_type:city&amp;title=Kensington+and+Chelsea">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°30′07″N
              </span>
              <span class="longitude">
               0°11′41″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5020°N 0.1947°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5020; -0.1947
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Kensington and Chelsea
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          3
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/Royal_Borough_of_Kingston_upon_Thames" title="Royal Borough of Kingston upon Thames">
           Kingston upon Thames
          </a>
         </td>
         <td>
         </td>
         <td>
          <a class="mw-redirect" href="/wiki/Royal_borough" title="Royal borough">
           Royal
          </a>
         </td>
         <td>
          <a href="/wiki/Kingston_upon_Thames_London_Borough_Council" title="Kingston upon Thames London Borough Council">
           Kingston upon Thames London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Liberal_Democrats_(UK)" title="Liberal Democrats (UK)">
           Liberal Democrat
          </a>
         </td>
         <td>
          <a href="/wiki/Kingston_upon_Thames_Guildhall" title="Kingston upon Thames Guildhall">
           Guildhall
          </a>
          , High Street
         </td>
         <td>
          14.38
         </td>
         <td>
          166,793
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4085_N_0.3064_W_region:GB_type:city&amp;title=Kingston+upon+Thames">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°24′31″N
              </span>
              <span class="longitude">
               0°18′23″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.4085°N 0.3064°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.4085; -0.3064
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Kingston upon Thames
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          16
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Lambeth" title="London Borough of Lambeth">
           Lambeth
          </a>
         </td>
         <td>
          <img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/>
          <span style="display:none">
           Y
          </span>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Lambeth_London_Borough_Council" title="Lambeth London Borough Council">
           Lambeth London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a href="/wiki/Lambeth_Town_Hall" title="Lambeth Town Hall">
           Lambeth Town Hall
          </a>
          , Brixton Hill
         </td>
         <td>
          10.36
         </td>
         <td>
          314,242
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4607_N_0.1163_W_region:GB_type:city&amp;title=Lambeth">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°27′39″N
              </span>
              <span class="longitude">
               0°06′59″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.4607°N 0.1163°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.4607; -0.1163
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Lambeth
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          6
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Lewisham" title="London Borough of Lewisham">
           Lewisham
          </a>
         </td>
         <td>
          <img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/>
          <span style="display:none">
           Y
          </span>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Lewisham_London_Borough_Council" title="Lewisham London Borough Council">
           Lewisham London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Lewisham_Town_Hall&amp;action=edit&amp;redlink=1" title="Lewisham Town Hall (page does not exist)">
           Town Hall
          </a>
          , 1 Catford Road
         </td>
         <td>
          13.57
         </td>
         <td>
          286,180
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4452_N_0.0209_W_region:GB_type:city&amp;title=Lewisham">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°26′43″N
              </span>
            <span class="longitude">
               0°01′15″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.4452°N 0.0209°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.4452; -0.0209
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Lewisham
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          21
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Merton" title="London Borough of Merton">
           Merton
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Merton_London_Borough_Council" title="Merton London Borough Council">
           Merton London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Merton_Civic_Centre&amp;action=edit&amp;redlink=1" title="Merton Civic Centre (page does not exist)">
           Civic Centre
          </a>
          , London Road
         </td>
         <td>
          14.52
         </td>
         <td>
          203,223
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4014_N_0.1958_W_region:GB_type:city&amp;title=Merton">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°24′05″N
              </span>
              <span class="longitude">
               0°11′45″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.4014°N 0.1958°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.4014; -0.1958
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Merton
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          17
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Newham" title="London Borough of Newham">
           Newham
          </a>
         </td>
         <td>
          <sup class="reference" id="cite_ref-note2_4-2">
           <a href="#cite_note-note2-4">
            [note 3]
           </a>
          </sup>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Newham_London_Borough_Council" title="Newham London Borough Council">
           Newham London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Newham_Dockside&amp;action=edit&amp;redlink=1" title="Newham Dockside (page does not exist)">
           Newham Dockside
          </a>
          , 1000 Dockside Road
         </td>
         <td>
          13.98
         </td>
         <td>
          318,227
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5077_N_0.0469_E_region:GB_type:city&amp;title=Newham">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°30′28″N
              </span>
              <span class="longitude">
               0°02′49″E
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5077°N 0.0469°E
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5077; 0.0469
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Newham
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          27
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Redbridge" title="London Borough of Redbridge">
           Redbridge
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Redbridge_London_Borough_Council" title="Redbridge London Borough Council">
           Redbridge London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Redbridge_Town_Hall&amp;action=edit&amp;redlink=1" title="Redbridge Town Hall (page does not exist)">
           Town Hall
          </a>
          , 128-142 High Road
         </td>
         <td>
          21.78
         </td>
         <td>
          288,272
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.559_N_0.0741_E_region:GB_type:city&amp;title=Redbridge">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°33′32″N
              </span>
              <span class="longitude">
               0°04′27″E
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5590°N 0.0741°E
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5590; 0.0741
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Redbridge
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          26
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Richmond_upon_Thames" title="London Borough of Richmond upon Thames">
           Richmond upon Thames
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Richmond_upon_Thames_London_Borough_Council" title="Richmond upon Thames London Borough Council">
           Richmond upon Thames London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Liberal_Democrats_(UK)" title="Liberal Democrats (UK)">
           Liberal Democrat
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Richmond_upon_Thames_Civic_Centre&amp;action=edit&amp;redlink=1" title="Richmond upon Thames Civic Centre (page does not exist)">
           Civic Centre
          </a>
          , 44 York Street
         </td>
         <td>
          22.17
         </td>
         <td>
          191,365
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4479_N_0.326_W_region:GB_type:city&amp;title=Richmond+upon+Thames">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°26′52″N
              </span>
              <span class="longitude">
               0°19′34″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.4479°N 0.3260°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.4479; -0.3260
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Richmond upon Thames
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          15
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Southwark" title="London Borough of Southwark">
           Southwark
          </a>
         </td>
         <td>
          <img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/>
          <span style="display:none">
           Y
          </span>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Southwark_London_Borough_Council" title="Southwark London Borough Council">
           Southwark London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=160_Tooley_Street&amp;action=edit&amp;redlink=1" title="160 Tooley Street (page does not exist)">
           160 Tooley Street
          </a>
         </td>
         <td>
          11.14
         </td>
         <td>
          298,464
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5035_N_0.0804_W_region:GB_type:city&amp;title=Southwark">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°30′13″N
              </span>
              <span class="longitude">
               0°04′49″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5035°N 0.0804°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5035; -0.0804
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Southwark
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          7
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Sutton" title="London Borough of Sutton">
           Sutton
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Sutton_London_Borough_Council" title="Sutton London Borough Council">
           Sutton London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Liberal_Democrats_(UK)" title="Liberal Democrats (UK)">
           Liberal Democrat
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Sutton_Civic_Offices&amp;action=edit&amp;redlink=1" title="Sutton Civic Offices (page does not exist)">
           Civic Offices
          </a>
          , St Nicholas Way
         </td>
         <td>
          16.93
         </td>
         <td>
          195,914
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.3618_N_0.1945_W_region:GB_type:city&amp;title=Sutton">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°21′42″N
              </span>
              <span class="longitude">
               0°11′40″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.3618°N 0.1945°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.3618; -0.1945
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Sutton
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          18
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Tower_Hamlets" title="London Borough of Tower Hamlets">
           Tower Hamlets
          </a>
         </td>
         <td>
          <img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/>
          <span style="display:none">
           Y
          </span>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Tower_Hamlets_London_Borough_Council" title="Tower Hamlets London Borough Council">
           Tower Hamlets London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Tower_Hamlets_Town_Hall&amp;action=edit&amp;redlink=1" title="Tower Hamlets Town Hall (page does not exist)">
           Town Hall
          </a>
          , Mulberry Place, 5 Clove Crescent
         </td>
         <td>
          7.63
         </td>
         <td>
          272,890
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5099_N_0.0059_W_region:GB_type:city&amp;title=Tower+Hamlets">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°30′36″N
              </span>
              <span class="longitude">
               0°00′21″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5099°N 0.0059°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5099; -0.0059
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Tower Hamlets
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          8
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Waltham_Forest" title="London Borough of Waltham Forest">
           Waltham Forest
          </a>
         </td>
         <td>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Waltham_Forest_London_Borough_Council" title="Waltham Forest London Borough Council">
           Waltham Forest London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">
           Labour
          </a>
         </td>
         <td>
          <a href="/wiki/Waltham_Forest_Town_Hall" title="Waltham Forest Town Hall">
           Waltham Forest Town Hall
          </a>
          , Forest Road
         </td>
         <td>
          14.99
         </td>
         <td>
          265,797
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5908_N_0.0134_W_region:GB_type:city&amp;title=Waltham+Forest">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°35′27″N
              </span>
              <span class="longitude">
               0°00′48″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5908°N 0.0134°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5908; -0.0134
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Waltham Forest
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          28
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/London_Borough_of_Wandsworth" title="London Borough of Wandsworth">
           Wandsworth
          </a>
         </td>
         <td>
          <img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/>
          <span style="display:none">
           Y
          </span>
         </td>
         <td>
         </td>
         <td>
          <a href="/wiki/Wandsworth_London_Borough_Council" title="Wandsworth London Borough Council">
           Wandsworth London Borough Council
          </a>
         </td>
         <td>
          <a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">
           Conservative
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Wandsworth_Town_Hall&amp;action=edit&amp;redlink=1" title="Wandsworth Town Hall (page does not exist)">
           The Town Hall
          </a>
          ,
          <a href="/wiki/Wandsworth_High_Street" title="Wandsworth High Street">
           Wandsworth High Street
          </a>
         </td>
         <td>
          13.23
         </td>
         <td>
          310,516
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4567_N_0.191_W_region:GB_type:city&amp;title=Wandsworth">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°27′24″N
              </span>
              <span class="longitude">
               0°11′28″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.4567°N 0.1910°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.4567; -0.1910
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Wandsworth
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          5
         </td>
        </tr>
        <tr>
         <td>
          <a href="/wiki/City_of_Westminster" title="City of Westminster">
           Westminster
          </a>
         </td>
         <td>
          <img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/>
          <span style="display:none">
           Y
          </span>
         </td>
         <td>
          <a href="/wiki/City_status_in_the_United_Kingdom" title="City status in the United Kingdom">
           City
          </a>
         </td>
         <td>
          <a href="/wiki/Westminster_City_Council" title="Westminster City Council">
           Westminster City Council
          </a>
         </td>
         <td>
          <a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">
           Conservative
          </a>
         </td>
         <td>
          <a class="new" href="/w/index.php?title=Westminster_City_Hall&amp;action=edit&amp;redlink=1" title="Westminster City Hall (page does not exist)">
           Westminster City Hall
          </a>
          , 64 Victoria Street
         </td>
         <td>
          8.29
         </td>
         <td>
          226,841
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4973_N_0.1372_W_region:GB_type:city&amp;title=Westminster">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°29′50″N
              </span>
              <span class="longitude">
               0°08′14″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.4973°N 0.1372°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.4973; -0.1372
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                Westminster
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          2
         </td>
        </tr>
       </tbody>
      </table>
      <h2>
       <span class="mw-headline" id="City_of_London">
        City of London
       </span>
       <span class="mw-editsection">
        <span class="mw-editsection-bracket">
         [
        </span>
        <a href="/w/index.php?title=List_of_London_boroughs&amp;action=edit&amp;section=2" title="Edit section: City of London">
         edit
        </a>
        <span class="mw-editsection-bracket">
         ]
        </span>
       </span>
      </h2>
      <p>
       The
       <a href="/wiki/City_of_London" title="City of London">
        City of London
       </a>
       is the 33rd principal division of Greater London but it is not a London borough.
      </p>
      <table class="wikitable sortable" style="font-size:95%" width="100%">
       <tbody>
        <tr>
         <th width="100px">
          <i>
           ‘Borough’
          </i>
         </th>
         <th>
          Inner
         </th>
         <th width="100px">
          Status
         </th>
         <th>
          Local authority
         </th>
         <th>
          Political control
         </th>
         <th width="120px">
          Headquarters
         </th>
         <th>
          Area
          <br/>
          (sq mi)
         </th>
         <th>
          Population
          <br/>
          (2011 est)
         </th>
         <th width="20px">
          Co-ordinates
         </th>
         <th>
          <span style="background:#67BCD3">
           Nr. in
           <br/>
           map
          </span>
         </th>
        </tr>
        <tr>
         <td>
          <a href="/wiki/City_of_London" title="City of London">
           City of London
          </a>
         </td>
         <td>
          (
          <img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/>
          <span style="display:none">
           Y
          </span>
          )
          <br/>
          <sup class="reference" id="cite_ref-6">
           <a href="#cite_note-6">
            [note 5]
           </a>
          </sup>
         </td>
         <td>
          <i>
           <a href="/wiki/Sui_generis" title="Sui generis">
            Sui generis
           </a>
          </i>
          ;
          <br/>
          <a href="/wiki/City_status_in_the_United_Kingdom" title="City status in the United Kingdom">
           City
          </a>
          ;
          <br/>
          <a href="/wiki/Ceremonial_counties_of_England" title="Ceremonial counties of England">
           Ceremonial county
          </a>
         </td>
         <td>
          <a class="mw-redirect" href="/wiki/Corporation_of_London" title="Corporation of London">
           Corporation of London
          </a>
          ;
          <br/>
          <a href="/wiki/Inner_Temple" title="Inner Temple">
           Inner Temple
          </a>
          ;
          <br/>
          <a href="/wiki/Middle_Temple" title="Middle Temple">
           Middle Temple
          </a>
         </td>
         <td>
          ?
         </td>
         <td>
          <a href="/wiki/Guildhall,_London" title="Guildhall, London">
           Guildhall
          </a>
         </td>
         <td>
          1.12
         </td>
         <td>
          7,000
         </td>
         <td>
          <span class="plainlinks nourlexpansion">
           <a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5155_N_0.0922_W_region:GB_type:city&amp;title=City+of+London">
            <span class="geo-nondefault">
             <span class="geo-dms" title="Maps, aerial photos, and other data for this location">
              <span class="latitude">
               51°30′56″N
              </span>
              <span class="longitude">
               0°05′32″W
              </span>
             </span>
            </span>
            <span class="geo-multi-punct">
             ﻿ / ﻿
            </span>
            <span class="geo-default">
             <span class="vcard">
              <span class="geo-dec" title="Maps, aerial photos, and other data for this location">
               51.5155°N 0.0922°W
              </span>
              <span style="display:none">
               ﻿ /
               <span class="geo">
                51.5155; -0.0922
               </span>
              </span>
              <span style="display:none">
               ﻿ (
               <span class="fn org">
                City of London
               </span>
               )
              </span>
             </span>
            </span>
           </a>
          </span>
         </td>
         <td>
          1
         </td>
        </tr>
       </tbody>
      </table>
      <table class="noprint infobox" id="GeoGroup" style="width: 23em; font-size: 88%; line-height: 1.5em">
       <tbody>
        <tr>
         <td>
          <b>
           Map all coordinates using:
          </b>
          <a class="external text" href="//tools.wmflabs.org/osm4wiki/cgi-bin/wiki/wiki-osm.pl?project=en&amp;article=List_of_London_boroughs">
           OpenStreetMap
          </a>
         </td>
        </tr>
        <tr>
         <td>
          <b>
           Download coordinates as:
          </b>
          <a class="external text" href="//tools.wmflabs.org/kmlexport?article=List_of_London_boroughs">
           KML
          </a>
          <b>
           ·
          </b>
          <a class="external text" href="http://tripgang.com/kml2gpx/http%3A%2F%2Ftools.wmflabs.org%2Fkmlexport%3Farticle%3DList_of_London_boroughs?gpx=1" rel="nofollow">
           GPX
          </a>
         </td>
        </tr>
       </tbody>
      </table>
      <h2>
       <span class="mw-headline" id="See_also">
        See also
       </span>
       <span class="mw-editsection">
        <span class="mw-editsection-bracket">
         [
        </span>
        <a href="/w/index.php?title=List_of_London_boroughs&amp;action=edit&amp;section=3" title="Edit section: See also">
         edit
        </a>
        <span class="mw-editsection-bracket">
         ]
        </span>
       </span>
      </h2>
      <table class="metadata mbox-small" role="presentation" style="background-color:#f9f9f9;border:1px solid #aaa;color:#000">
       <tbody>
        <tr>
         <td class="mbox-image">
          <img alt="icon" data-file-height="48" data-file-width="48" decoding="async" height="30" src="//upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Office-book.svg/30px-Office-book.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Office-book.svg/45px-Office-book.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Office-book.svg/60px-Office-book.svg.png 2x" width="30"/>
         </td>
         <td class="mbox-text plainlist">
          <ul style="font-weight: bold">
           <li>
            <a href="/wiki/Book:London" title="Book:London">
             Book: London
            </a>
           </li>
          </ul>
         </td>
        </tr>
       </tbody>
      </table>
      <ul>
       <li>
        <a class="mw-redirect" href="/wiki/Political_make-up_of_London_borough_councils" title="Political make-up of London borough councils">
         Political make-up of London borough councils
        </a>
       </li>
       <li>
        <a href="/wiki/List_of_areas_of_London" title="List of areas of London">
         List of areas of London
        </a>
       </li>
       <li>
        <a href="/wiki/Subdivisions_of_England" title="Subdivisions of England">
         Subdivisions of England
        </a>
       </li>
      </ul>
      <h2>
       <span class="mw-headline" id="Notes">
        Notes
       </span>
       <span class="mw-editsection">
        <span class="mw-editsection-bracket">
         [
        </span>
        <a href="/w/index.php?title=List_of_London_boroughs&amp;action=edit&amp;section=4" title="Edit section: Notes">
         edit
        </a>
        <span class="mw-editsection-bracket">
         ]
        </span>
       </span>
      </h2>
      <div class="reflist" style="list-style-type: decimal;">
       <div class="mw-references-wrap">
        <ol class="references">
         <li id="cite_note-2">
          <span class="mw-cite-backlink">
           <b>
            <a href="#cite_ref-2">
             ^
            </a>
           </b>
          </span>
          <span class="reference-text">
           Renamed from London Borough of Barking 1 January 1980.
           <cite class="citation magazine" id="CITEREFGazette48021">
            <a class="external text" href="https://www.thegazette.co.uk/London/issue/48021/page/15280" rel="nofollow">
             "No. 48021"
            </a>
            .
            <i>
             <a href="/wiki/The_London_Gazette" title="The London Gazette">
              The London Gazette
             </a>
            </i>
            . 4 December 1979. p. 15280.
           </cite>
           <span class="Z3988" title="ctx_ver=Z39.88-2004&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.jtitle=The+London+Gazette&amp;rft.atitle=No.+48021&amp;rft.pages=15280&amp;rft.date=1979-12-04&amp;rft_id=https%3A%2F%2Fwww.thegazette.co.uk%2FLondon%2Fissue%2F48021%2Fpage%2F15280&amp;rfr_id=info%3Asid%2Fen.wikipedia.org%3AList+of+London+boroughs"/>
           <style data-mw-deduplicate="TemplateStyles:r886058088">
            .mw-parser-output cite.citation{font-style:inherit}.mw-parser-output .citation q{quotes:"\"""\"""'""'"}.mw-parser-output .citation .cs1-lock-free a{background:url("//upload.wikimedia.org/wikipedia/commons/thumb/6/65/Lock-green.svg/9px-Lock-green.svg.png")no-repeat;background-position:right .1em center}.mw-parser-output .citation .cs1-lock-limited a,.mw-parser-output .citation .cs1-lock-registration a{background:url("//upload.wikimedia.org/wikipedia/commons/thumb/d/d6/Lock-gray-alt-2.svg/9px-Lock-gray-alt-2.svg.png")no-repeat;background-position:right .1em center}.mw-parser-output .citation .cs1-lock-subscription a{background:url("//upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Lock-red-alt-2.svg/9px-Lock-red-alt-2.svg.png")no-repeat;background-position:right .1em center}.mw-parser-output .cs1-subscription,.mw-parser-output .cs1-registration{color:#555}.mw-parser-output .cs1-subscription span,.mw-parser-output .cs1-registration span{border-bottom:1px dotted;cursor:help}.mw-parser-output .cs1-ws-icon a{background:url("//upload.wikimedia.org/wikipedia/commons/thumb/4/4c/Wikisource-logo.svg/12px-Wikisource-logo.svg.png")no-repeat;background-position:right .1em center}.mw-parser-output code.cs1-code{color:inherit;background:inherit;border:inherit;padding:inherit}.mw-parser-output .cs1-hidden-error{display:none;font-size:100%}.mw-parser-output .cs1-visible-error{font-size:100%}.mw-parser-output .cs1-maint{display:none;color:#33aa33;margin-left:0.3em}.mw-parser-output .cs1-subscription,.mw-parser-output .cs1-registration,.mw-parser-output .cs1-format{font-size:95%}.mw-parser-output .cs1-kern-left,.mw-parser-output .cs1-kern-wl-left{padding-left:0.2em}.mw-parser-output .cs1-kern-right,.mw-parser-output .cs1-kern-wl-right{padding-right:0.2em}
           </style>
          </span>
         </li>
         <li id="cite_note-3">
          <span class="mw-cite-backlink">
           <b>
            <a href="#cite_ref-3">
             ^
            </a>
           </b>
          </span>
          <span class="reference-text">
           Royal borough from 2012
          </span>
         </li>
         <li id="cite_note-note2-4">
          <span class="mw-cite-backlink">
           ^
           <a href="#cite_ref-note2_4-0">
            <sup>
             <i>
              <b>
               a
              </b>
             </i>
            </sup>
           </a>
           <a href="#cite_ref-note2_4-1">
            <sup>
             <i>
              <b>
               b
              </b>
             </i>
            </sup>
           </a>
           <a href="#cite_ref-note2_4-2">
            <sup>
             <i>
              <b>
               c
              </b>
             </i>
            </sup>
           </a>
          </span>
          <span class="reference-text">
           Haringey and Newham are Inner London for statistics; Greenwich is Outer London for statistics
          </span>
         </li>
         <li id="cite_note-5">
          <span class="mw-cite-backlink">
           <b>
            <a href="#cite_ref-5">
             ^
            </a>
           </b>
          </span>
          <span class="reference-text">
           Renamed from London Borough of Hammersmith 1 April 1979.
           <cite class="citation magazine" id="CITEREFGazette47771">
            <a class="external text" href="https://www.thegazette.co.uk/London/issue/47771/page/2095" rel="nofollow">
             "No. 47771"
            </a>
            .
            <i>
             <a href="/wiki/The_London_Gazette" title="The London Gazette">
              The London Gazette
             </a>
            </i>
            . 13 February 1979. p. 2095.
           </cite>
           <span class="Z3988" title="ctx_ver=Z39.88-2004&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=article&amp;rft.jtitle=The+London+Gazette&amp;rft.atitle=No.+47771&amp;rft.pages=2095&amp;rft.date=1979-02-13&amp;rft_id=https%3A%2F%2Fwww.thegazette.co.uk%2FLondon%2Fissue%2F47771%2Fpage%2F2095&amp;rfr_id=info%3Asid%2Fen.wikipedia.org%3AList+of+London+boroughs"/>
           <link href="mw-data:TemplateStyles:r886058088" rel="mw-deduplicated-inline-style"/>
          </span>
         </li>
         <li id="cite_note-6">
          <span class="mw-cite-backlink">
           <b>
            <a href="#cite_ref-6">
             ^
            </a>
           </b>
          </span>
          <span class="reference-text">
           The City of London was not part of the
           <a href="/wiki/County_of_London" title="County of London">
            County of London
           </a>
           and is not a London Borough but can be counted to
           <a href="/wiki/Inner_London" title="Inner London">
            Inner London
           </a>
           .
          </span>
         </li>
        </ol>
       </div>
      </div>
      <h2>
       <span class="mw-headline" id="References">
        References
       </span>
       <span class="mw-editsection">
        <span class="mw-editsection-bracket">
         [
        </span>
        <a href="/w/index.php?title=List_of_London_boroughs&amp;action=edit&amp;section=5" title="Edit section: References">
         edit
        </a>
        <span class="mw-editsection-bracket">
         ]
        </span>
       </span>
      </h2>
      <div class="reflist" style="list-style-type: decimal;">
       <div class="mw-references-wrap">
        <ol class="references">
         <li id="cite_note-1">
          <span class="mw-cite-backlink">
           <b>
            <a href="#cite_ref-1">
             ^
            </a>
           </b>
          </span>
          <span class="reference-text">
           <cite class="citation web">
            ONS (2 July 2010).
            <a class="external text" href="https://webarchive.nationalarchives.gov.uk/20160107070948/http://www.ons.gov.uk/ons/publications/re-reference-tables.html" rel="nofollow">
             "Release Edition Reference Tables"
            </a>
            .
            <i>
             Webarchive.nationalarchives.gov.uk
            </i>
            <span class="reference-accessdate">
             . Retrieved
             <span class="nowrap">
              5 February
             </span>
             2019
            </span>
            .
           </cite>
           <span class="Z3988" title="ctx_ver=Z39.88-2004&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal&amp;rft.genre=unknown&amp;rft.jtitle=Webarchive.nationalarchives.gov.uk&amp;rft.atitle=Release+Edition+Reference+Tables&amp;rft.date=2010-07-02&amp;rft.au=ONS&amp;rft_id=https%3A%2F%2Fwebarchive.nationalarchives.gov.uk%2F20160107070948%2Fhttp%3A%2F%2Fwww.ons.gov.uk%2Fons%2Fpublications%2Fre-reference-tables.html&amp;rfr_id=info%3Asid%2Fen.wikipedia.org%3AList+of+London+boroughs"/>
           <link href="mw-data:TemplateStyles:r886058088" rel="mw-deduplicated-inline-style"/>
          </span>
         </li>
        </ol>
       </div>
      </div>
      <h2>
       <span class="mw-headline" id="External_links">
        External links
       </span>
       <span class="mw-editsection">
        <span class="mw-editsection-bracket">
         [
        </span>
        <a href="/w/index.php?title=List_of_London_boroughs&amp;action=edit&amp;section=6" title="Edit section: External links">
         edit
        </a>
        <span class="mw-editsection-bracket">
         ]
        </span>
       </span>
      </h2>
      <ul>
       <li>
        <a class="external text" href="https://web.archive.org/web/20101010011530/http://londoncouncils.gov.uk/londonlocalgovernment/londonboroughs.htm" rel="nofollow">
         London Councils: List of inner/outer London boroughs
        </a>
       </li>
       <li>
        <a class="external text" href="http://londonboroughsmap.co.uk/" rel="nofollow">
         London Boroughs Map
        </a>
       </li>
      </ul>
      <div aria-labelledby="Governance_of_Greater_London" class="navbox" role="navigation" style="padding:3px">
       <table class="nowraplinks hlist collapsible collapsed navbox-inner" style="border-spacing:0;background:transparent;color:inherit">
        <tbody>
         <tr>
          <th class="navbox-title" colspan="2" scope="col">
           <div class="plainlinks hlist navbar mini">
            <ul>
             <li class="nv-view">
              <a href="/wiki/Template:Governance_of_Greater_London" title="Template:Governance of Greater London">
               <abbr style=";;background:none transparent;border:none;-moz-box-shadow:none;-webkit-box-shadow:none;box-shadow:none; padding:0;" title="View this template">
                v
               </abbr>
              </a>
             </li>
             <li class="nv-talk">
              <a href="/wiki/Template_talk:Governance_of_Greater_London" title="Template talk:Governance of Greater London">
               <abbr style=";;background:none transparent;border:none;-moz-box-shadow:none;-webkit-box-shadow:none;box-shadow:none; padding:0;" title="Discuss this template">
                t
               </abbr>
              </a>
             </li>
             <li class="nv-edit">
              <a class="external text" href="//en.wikipedia.org/w/index.php?title=Template:Governance_of_Greater_London&amp;action=edit">
               <abbr style=";;background:none transparent;border:none;-moz-box-shadow:none;-webkit-box-shadow:none;box-shadow:none; padding:0;" title="Edit this template">
                e
               </abbr>
              </a>
             </li>
            </ul>
           </div>
           <div id="Governance_of_Greater_London" style="font-size:114%;margin:0 4em">
            Governance of
            <a href="/wiki/Greater_London" title="Greater London">
             Greater London
            </a>
           </div>
          </th>
         </tr>
         <tr>
          <td class="navbox-abovebelow" colspan="2">
           <div id="*_City_of_London&amp;#10;*_London">
            <ul>
             <li>
              <a href="/wiki/City_of_London" title="City of London">
               City of London
              </a>
             </li>
             <li>
              <a href="/wiki/London" title="London">
               London
              </a>
             </li>
            </ul>
           </div>
          </td>
         </tr>
         <tr>
          <th class="navbox-group" scope="row" style="width:1%">
           Regional
          </th>
          <td class="navbox-list navbox-odd" style="text-align:left;border-left-width:2px;border-left-style:solid;width:100%;padding:0px">
           <div style="padding:0em 0.25em">
            <ul>
             <li>
              <b>
               <a href="/wiki/Greater_London_Authority" title="Greater London Authority">
                Greater London Authority
               </a>
               :
              </b>
              <a href="/wiki/London_Assembly" title="London Assembly">
               London Assembly
              </a>
             </li>
             <li>
              <a href="/wiki/Mayor_of_London" title="Mayor of London">
               Mayor of London
              </a>
             </li>
            </ul>
           </div>
          </td>
         </tr>
         <tr>
          <th class="navbox-group" scope="row" style="width:1%">
           <a href="/wiki/London_boroughs" title="London boroughs">
            Boroughs
           </a>
          </th>
          <td class="navbox-list navbox-even" style="text-align:left;border-left-width:2px;border-left-style:solid;width:100%;padding:0px">
           <div style="padding:0em 0.25em">
            <ul>
             <li>
              <b>
               <a href="/wiki/London_Councils" title="London Councils">
                London Councils
               </a>
               :
              </b>
              <a href="/wiki/London_Borough_of_Barking_and_Dagenham" title="London Borough of Barking and Dagenham">
               Barking and Dagenham
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Barnet" title="London Borough of Barnet">
               Barnet
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Bexley" title="London Borough of Bexley">
               Bexley
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Brent" title="London Borough of Brent">
               Brent
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Bromley" title="London Borough of Bromley">
               Bromley
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Camden" title="London Borough of Camden">
               Camden
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Croydon" title="London Borough of Croydon">
               Croydon
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Ealing" title="London Borough of Ealing">
               Ealing
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Enfield" title="London Borough of Enfield">
               Enfield
              </a>
             </li>
             <li>
              <a href="/wiki/Royal_Borough_of_Greenwich" title="Royal Borough of Greenwich">
               Greenwich
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Hackney" title="London Borough of Hackney">
               Hackney
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Hammersmith_and_Fulham" title="London Borough of Hammersmith and Fulham">
               Hammersmith and Fulham
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Haringey" title="London Borough of Haringey">
               Haringey
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Harrow" title="London Borough of Harrow">
               Harrow
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Havering" title="London Borough of Havering">
               Havering
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Hillingdon" title="London Borough of Hillingdon">
               Hillingdon
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Hounslow" title="London Borough of Hounslow">
               Hounslow
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Islington" title="London Borough of Islington">
               Islington
              </a>
             </li>
             <li>
              <a href="/wiki/Royal_Borough_of_Kensington_and_Chelsea" title="Royal Borough of Kensington and Chelsea">
               Kensington and Chelsea
              </a>
             </li>
             <li>
              <a href="/wiki/Royal_Borough_of_Kingston_upon_Thames" title="Royal Borough of Kingston upon Thames">
               Kingston upon Thames
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Lambeth" title="London Borough of Lambeth">
               Lambeth
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Lewisham" title="London Borough of Lewisham">
               Lewisham
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Merton" title="London Borough of Merton">
               Merton
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Newham" title="London Borough of Newham">
               Newham
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Redbridge" title="London Borough of Redbridge">
               Redbridge
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Richmond_upon_Thames" title="London Borough of Richmond upon Thames">
               Richmond upon Thames
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Southwark" title="London Borough of Southwark">
               Southwark
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Sutton" title="London Borough of Sutton">
               Sutton
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Tower_Hamlets" title="London Borough of Tower Hamlets">
               Tower Hamlets
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Waltham_Forest" title="London Borough of Waltham Forest">
               Waltham Forest
              </a>
             </li>
             <li>
              <a href="/wiki/London_Borough_of_Wandsworth" title="London Borough of Wandsworth">
               Wandsworth
              </a>
             </li>
             <li>
              <a href="/wiki/City_of_Westminster" title="City of Westminster">
               Westminster
              </a>
             </li>
            </ul>
           </div>
          </td>
         </tr>
         <tr>
          <th class="navbox-group" scope="row" style="width:1%">
           Ceremonial
          </th>
          <td class="navbox-list navbox-odd" style="text-align:left;border-left-width:2px;border-left-style:solid;width:100%;padding:0px">
           <div style="padding:0em 0.25em">
            <ul>
             <li>
              <a href="/wiki/Lord_Mayor_of_London" title="Lord Mayor of London">
               Lord Mayor of the City of London
              </a>
             </li>
             <li>
              <a class="mw-redirect" href="/wiki/Lord_Lieutenant_of_Greater_London" title="Lord Lieutenant of Greater London">
               Lord Lieutenant of Greater London
              </a>
             </li>
             <li>
              <a href="/wiki/High_Sheriff_of_Greater_London" title="High Sheriff of Greater London">
               High Sheriff of Greater London
              </a>
             </li>
            </ul>
           </div>
          </td>
         </tr>
         <tr>
          <th class="navbox-group" scope="row" style="width:1%">
           <a href="/wiki/History_of_local_government_in_London" title="History of local government in London">
            Historical
           </a>
          </th>
          <td class="navbox-list navbox-even" style="text-align:left;border-left-width:2px;border-left-style:solid;width:100%;padding:0px">
           <div style="padding:0em 0.25em">
            <ul>
             <li>
              <a href="/wiki/Metropolitan_Board_of_Works" title="Metropolitan Board of Works">
               Metropolitan Board of Works
              </a>
              <span style="font-size:85%;">
               (MBW) 1855–1889
              </span>
             </li>
             <li>
              <a href="/wiki/London_County_Council" title="London County Council">
               London County Council
              </a>
              <span style="font-size:85%;">
               (LCC) 1889–1965
              </span>
             </li>
             <li>
              <a href="/wiki/Greater_London_Council" title="Greater London Council">
               Greater London Council
              </a>
              <span style="font-size:85%;">
               (GLC) 1965–1986
              </span>
             </li>
             <li>
              <a href="/wiki/List_of_heads_of_London_government" title="List of heads of London government">
               Leaders
              </a>
             </li>
             <li>
              <a href="/wiki/Sheriffs_of_the_City_of_London" title="Sheriffs of the City of London">
               Sheriffs of the City of London
              </a>
             </li>
            </ul>
           </div>
          </td>
         </tr>
        </tbody>
       </table>
      </div>
      <!-- 
NewPP limit report
Parsed by mw1269
Cached time: 20190428014242
Cache expiry: 2592000
Dynamic content: false
CPU time usage: 0.324 seconds
Real time usage: 0.395 seconds
Preprocessor visited node count: 5142/1000000
Preprocessor generated node count: 0/1500000
Post‐expand include size: 79976/2097152 bytes
Template argument size: 991/2097152 bytes
Highest expansion depth: 12/40
Expensive parser function count: 2/500
Unstrip recursion depth: 1/20
Unstrip post‐expand size: 10101/5000000 bytes
Number of Wikibase entities loaded: 0/400
Lua time usage: 0.111/10.000 seconds
Lua memory usage: 2.75 MB/50 MB
-->
      <!--
Transclusion expansion time report (%,ms,calls,template)
100.00%  279.718      1 -total
 41.33%  115.594      2 Template:Reflist
 32.47%   90.827      2 Template:London_Gazette
 30.02%   83.982      2 Template:Cite_magazine
 20.77%   58.105     33 Template:Coord
  9.59%   26.818     33 Template:English_district_control
  9.47%   26.499      1 Template:Use_dmy_dates
  5.78%   16.180      1 Template:London
  4.97%   13.913      2 Template:DMCA
  4.23%   11.829      2 Template:Dated_maintenance_category
-->
      <!-- Saved in parser cache with key enwiki:pcache:idhash:28092685-0!canonical and timestamp 20190428014242 and revision id 881899861
 -->
     </div>
     <noscript>
      <img alt="" height="1" src="//en.wikipedia.org/wiki/Special:CentralAutoLogin/start?type=1x1" style="border: none; position: absolute;" title="" width="1"/>
     </noscript>
    </div>
    <div class="printfooter">
     Retrieved from "
     <a dir="ltr" href="https://en.wikipedia.org/w/index.php?title=List_of_London_boroughs&amp;oldid=881899861">
      https://en.wikipedia.org/w/index.php?title=List_of_London_boroughs&amp;oldid=881899861
     </a>
     "
    </div>
    <div class="catlinks" data-mw="interface" id="catlinks">
     <div class="mw-normal-catlinks" id="mw-normal-catlinks">
      <a href="/wiki/Help:Category" title="Help:Category">
       Categories
      </a>
      :
      <ul>
       <li>
        <a href="/wiki/Category:London_boroughs" title="Category:London boroughs">
         London boroughs
         <li>
        <a href="/wiki/Category:Lists_of_places_in_London" title="Category:Lists of places in London">
         Lists of places in London
        </a>
       </li>
      </ul>
     </div>
     <div class="mw-hidden-catlinks mw-hidden-cats-hidden" id="mw-hidden-catlinks">
      Hidden categories:
      <ul>
       <li>
        <a href="/wiki/Category:Use_dmy_dates_from_August_2015" title="Category:Use dmy dates from August 2015">
         Use dmy dates from August 2015
        </a>
       </li>
       <li>
        <a href="/wiki/Category:Use_British_English_from_August_2015" title="Category:Use British English from August 2015">
         Use British English from August 2015
        </a>
       </li>
       <li>
        <a href="/wiki/Category:Lists_of_coordinates" title="Category:Lists of coordinates">
         Lists of coordinates
        </a>
       </li>
       <li>
        <a href="/wiki/Category:Geographic_coordinate_lists" title="Category:Geographic coordinate lists">
         Geographic coordinate lists
        </a>
       </li>
       <li>
        <a href="/wiki/Category:Articles_with_Geo" title="Category:Articles with Geo">
         Articles with Geo
        </a>
       </li>
      </ul>
     </div>
    </div>
    <div class="visualClear"/>
   </div>
  </div>
  <div id="mw-navigation">
   <h2>
    Navigation menu
   </h2>
   <div id="mw-head">
    <div aria-labelledby="p-personal-label" id="p-personal" role="navigation">
     <h3 id="p-personal-label">
      Personal tools
     </h3>
     <ul>
      <li id="pt-anonuserpage">
       Not logged in
      </li>
      <li id="pt-anontalk">
       <a accesskey="n" href="/wiki/Special:MyTalk" title="Discussion about edits from this IP address [n]">
        Talk
       </a>
      </li>
      <li id="pt-anoncontribs">
       <a accesskey="y" href="/wiki/Special:MyContributions" title="A list of edits made from this IP address [y]">
        Contributions
       </a>
      </li>
      <li id="pt-createaccount">
       <a href="/w/index.php?title=Special:CreateAccount&amp;returnto=List+of+London+boroughs" title="You are encouraged to create an account and log in; however, it is not mandatory">
        Create account
       </a>
      </li>
      <li id="pt-login">
       <a accesskey="o" href="/w/index.php?title=Special:UserLogin&amp;returnto=List+of+London+boroughs" title="You're encouraged to log in; however, it's not mandatory. [o]">
        Log in
       </a>
      </li>
     </ul>
    </div>
    <div id="left-navigation">
     <div aria-labelledby="p-namespaces-label" class="vectorTabs" id="p-namespaces" role="navigation">
      <h3 id="p-namespaces-label">
       Namespaces
      </h3>
      <ul>
       <li class="selected" id="ca-nstab-main">
        <span>
         <a accesskey="c" href="/wiki/List_of_London_boroughs" title="View the content page [c]">
          Article
         </a>
        </span>
       </li>
       <li id="ca-talk">
        <span>
         <a accesskey="t" href="/wiki/Talk:List_of_London_boroughs" rel="discussion" title="Discussion about the content page [t]">
          Talk
         </a>
        </span>
       </li>
      </ul>
     </div>
     <div aria-labelledby="p-variants-label" class="vectorMenu emptyPortlet" id="p-variants" role="navigation">
      <input aria-labelledby="p-variants-label" class="vectorMenuCheckbox" type="checkbox"/>
      <h3 id="p-variants-label">
       <span>
        Variants
       </span>
      </h3>
      <ul class="menu">
      </ul>
     </div>
    </div>
    <div id="right-navigation">
     <div aria-labelledby="p-views-label" class="vectorTabs" id="p-views" role="navigation">
      <h3 id="p-views-label">
       Views
      </h3>
      <ul>
       <li class="collapsible selected" id="ca-view">
        <span>
         <a href="/wiki/List_of_London_boroughs">
          Read
         </a>
        </span>
       </li>
       <li class="collapsible" id="ca-edit">
        <span>
         <a accesskey="e" href="/w/index.php?title=List_of_London_boroughs&amp;action=edit" title="Edit this page [e]">
          Edit
         </a>
        </span>
       </li>
       <li class="collapsible" id="ca-history">
        <span>
         <a accesskey="h" href="/w/index.php?title=List_of_London_boroughs&amp;action=history" title="Past revisions of this page [h]">
          View history
         </a>
        </span>
       </li>
      </ul>
     </div>
     <div aria-labelledby="p-cactions-label" class="vectorMenu emptyPortlet" id="p-cactions" role="navigation">
      <input aria-labelledby="p-cactions-label" class="vectorMenuCheckbox" type="checkbox"/>
      <h3 id="p-cactions-label">
       <span>
        More
       </span>
      </h3>
      <ul class="menu">
      </ul>
     </div>
     <div id="p-search" role="search">
      <h3>
       <label for="searchInput">
        Search
       </label>
      </h3>
      <form action="/w/index.php" id="searchform">
       <div id="simpleSearch">
        <input accesskey="f" id="searchInput" name="search" placeholder="Search Wikipedia" title="Search Wikipedia [f]" type="search"/>
        <input name="title" type="hidden" value="Special:Search"/>
        <input class="searchButton mw-fallbackSearchButton" id="mw-searchButton" name="fulltext" title="Search Wikipedia for this text" type="submit" value="Search"/>
        <input class="searchButton" id="searchButton" name="go" title="Go to a page with this exact name if it exists" type="submit" value="Go"/>
       </div>
      </form>
     </div>
    </div>
   </div>
   <div id="mw-panel">
    <div id="p-logo" role="banner">
     <a class="mw-wiki-logo" href="/wiki/Main_Page" title="Visit the main page"/>
    </div>
    <div aria-labelledby="p-navigation-label" class="portal" id="p-navigation" role="navigation">
     <h3 id="p-navigation-label">
      Navigation
     </h3>
     <div class="body">
      <ul>
       <li id="n-mainpage-description">
        <a accesskey="z" href="/wiki/Main_Page" title="Visit the main page [z]">
         Main page
        </a>
       </li>
       <li id="n-contents">
        <a href="/wiki/Portal:Contents" title="Guides to browsing Wikipedia">
         Contents
        </a>
       </li>
       <li id="n-featuredcontent">
        <a href="/wiki/Portal:Featured_content" title="Featured content – the best of Wikipedia">
         Featured content
        </a>
       </li>
       <li id="n-currentevents">
        <a href="/wiki/Portal:Current_events" title="Find background information on current events">
         Current events
        </a>
       </li>
       <li id="n-randompage">
        <a accesskey="x" href="/wiki/Special:Random" title="Load a random article [x]">
         Random article
        </a>
       </li>
       <li id="n-sitesupport">
        <a href="https://donate.wikimedia.org/wiki/Special:FundraiserRedirector?utm_source=donate&amp;utm_medium=sidebar&amp;utm_campaign=C13_en.wikipedia.org&amp;uselang=en" title="Support us">
         Donate to Wikipedia
        </a>
       </li>
       <li id="n-shoplink">
        <a href="//shop.wikimedia.org" title="Visit the Wikipedia store">
         Wikipedia store
        </a>
       </li>
      </ul>
     </div>
    </div>
    <div aria-labelledby="p-interaction-label" class="portal" id="p-interaction" role="navigation">
     <h3 id="p-interaction-label">
      Interaction
     </h3>
     <div class="body">
      <ul>
       <li id="n-help">
        <a href="/wiki/Help:Contents" title="Guidance on how to use and edit Wikipedia">
         Help
        </a>
       </li>
       <li id="n-aboutsite">
        <a href="/wiki/Wikipedia:About" title="Find out about Wikipedia">
         About Wikipedia
        </a>
       </li>
       <li id="n-portal">
        <a href="/wiki/Wikipedia:Community_portal" title="About the project, what you can do, where to find things">
         Community portal
        </a>
       </li>
       <li id="n-recentchanges">
        <a accesskey="r" href="/wiki/Special:RecentChanges" title="A list of recent changes in the wiki [r]">
         Recent changes
        </a>
       </li>
       <li id="n-contactpage">
        <a href="//en.wikipedia.org/wiki/Wikipedia:Contact_us" title="How to contact Wikipedia">
         Contact page
        </a>
       </li>
      </ul>
     </div>
    </div>
    <div aria-labelledby="p-tb-label" class="portal" id="p-tb" role="navigation">
     <h3 id="p-tb-label">
      Tools
     </h3>
     <div class="body">
      <ul>
       <li id="t-whatlinkshere">
        <a accesskey="j" href="/wiki/Special:WhatLinksHere/List_of_London_boroughs" title="List of all English Wikipedia pages containing links to this page [j]">
         What links here
        </a>
       </li>
       <li id="t-recentchangeslinked">
        <a accesskey="k" href="/wiki/Special:RecentChangesLinked/List_of_London_boroughs" rel="nofollow" title="Recent changes in pages linked from this page [k]">
         Related changes
        </a>
       </li>
       <li id="t-upload">
        <a accesskey="u" href="/wiki/Wikipedia:File_Upload_Wizard" title="Upload files [u]">
         Upload file
        </a>
       </li>
       <li id="t-specialpages">
        <a accesskey="q" href="/wiki/Special:SpecialPages" title="A list of all special pages [q]">
         Special pages
        </a>
       </li>
       <li id="t-permalink">
        <a href="/w/index.php?title=List_of_London_boroughs&amp;oldid=881899861" title="Permanent link to this revision of the page">
         Permanent link
        </a>
       </li>
       <li id="t-info">
        <a href="/w/index.php?title=List_of_London_boroughs&amp;action=info" title="More information about this page">
         Page information
        </a>
       </li>
       <li id="t-wikibase">
        <a accesskey="g" href="https://www.wikidata.org/wiki/Special:EntityPage/Q6577004" title="Link to connected data repository item [g]">
         Wikidata item
        </a>
       </li>
       <li id="t-cite">
        <a href="/w/index.php?title=Special:CiteThisPage&amp;page=List_of_London_boroughs&amp;id=881899861" title="Information on how to cite this page">
         Cite this page
        </a>
       </li>
      </ul>
     </div>
    </div>
    <div aria-labelledby="p-coll-print_export-label" class="portal" id="p-coll-print_export" role="navigation">
     <h3 id="p-coll-print_export-label">
      Print/export
     </h3>
     <div class="body">
      <ul>
       <li id="coll-create_a_book">
        <a href="/w/index.php?title=Special:Book&amp;bookcmd=book_creator&amp;referer=List+of+London+boroughs">
         Create a book
        </a>
       </li>
       <li id="coll-download-as-rdf2latex">
        <a href="/w/index.php?title=Special:ElectronPdf&amp;page=List+of+London+boroughs&amp;action=show-download-screen">
         Download as PDF
        </a>
       </li>
       <li id="t-print">
        <a accesskey="p" href="/w/index.php?title=List_of_London_boroughs&amp;printable=yes" title="Printable version of this page [p]">
         Printable version
        </a>
       </li>
      </ul>
     </div>
    </div>
    <div aria-labelledby="p-lang-label" class="portal" id="p-lang" role="navigation">
     <h3 id="p-lang-label">
      Languages
     </h3>
     <div class="body">
      <ul>
       <li class="interlanguage-link interwiki-ru">
        <a class="interlanguage-link-target" href="https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BB%D0%BE%D0%BD%D0%B4%D0%BE%D0%BD%D1%81%D0%BA%D0%B8%D1%85_%D0%B1%D0%BE%D1%80%D0%BE" hreflang="ru" lang="ru" title="Список лондонских боро – Russian">
         Русский
        </a>
       </li>
      </ul>
      <div class="after-portlet after-portlet-lang">
       <span class="wb-langlinks-edit wb-langlinks-link">
        <a class="wbc-editpage" href="https://www.wikidata.org/wiki/Special:EntityPage/Q6577004#sitelinks-wikipedia" title="Edit interlanguage links">
         Edit links
        </a>
       </span>
      </div>
     </div>
    </div>
   </div>
  </div>
  <div id="footer" role="contentinfo">
   <ul id="footer-info">
    <li id="footer-info-lastmod">
     This page was last edited on 5 February 2019, at 14:50
     <span class="anonymous-show">
      (UTC)
     </span>
     .
    </li>
    <li id="footer-info-copyright">
     Text is available under the
     <a href="//en.wikipedia.org/wiki/Wikipedia:Text_of_Creative_Commons_Attribution-ShareAlike_3.0_Unported_License" rel="license">
      Creative Commons Attribution-ShareAlike License
     </a>
     <a href="//creativecommons.org/licenses/by-sa/3.0/" rel="license" style="display:none;"/>
     ;
additional terms may apply.  By using this site, you agree to the
     <a href="//foundation.wikimedia.org/wiki/Terms_of_Use">
      Terms of Use
     </a>
     and
     <a href="//foundation.wikimedia.org/wiki/Privacy_policy">
      Privacy Policy
     </a>
     . Wikipedia® is a registered trademark of the
     <a href="//www.wikimediafoundation.org/">
      Wikimedia Foundation, Inc.
     </a>
     , a non-profit organization.
    </li>
   </ul>
   <ul id="footer-places">
    <li id="footer-places-privacy">
     <a class="extiw" href="https://foundation.wikimedia.org/wiki/Privacy_policy" title="wmf:Privacy policy">
      Privacy policy
     </a>
    </li>
    <li id="footer-places-about">
     <a href="/wiki/Wikipedia:About" title="Wikipedia:About">
      About Wikipedia
     </a>
    </li>
    <li id="footer-places-disclaimer">
     <a href="/wiki/Wikipedia:General_disclaimer" title="Wikipedia:General disclaimer">
      Disclaimers
     </a>
    </li>
    <li id="footer-places-contact">
     <a href="//en.wikipedia.org/wiki/Wikipedia:Contact_us">
      Contact Wikipedia
     </a>
    </li>
    <li id="footer-places-developers">
     <a href="https://www.mediawiki.org/wiki/Special:MyLanguage/How_to_contribute">
      Developers
     </a>
    </li>
    <li id="footer-places-cookiestatement">
     <a href="https://foundation.wikimedia.org/wiki/Cookie_statement">
      Cookie statement
     </a>
    </li>
    <li id="footer-places-mobileview">
     <a class="noprint stopMobileRedirectToggle" href="//en.m.wikipedia.org/w/index.php?title=List_of_London_boroughs&amp;mobileaction=toggle_view_mobile">
      Mobile view
     </a>
    </li>
   </ul>
   <ul class="noprint" id="footer-icons">
    <li id="footer-copyrightico">
     <a href="https://wikimediafoundation.org/">
      <img alt="Wikimedia Foundation" height="31" src="/static/images/wikimedia-button.png" srcset="/static/images/wikimedia-button-1.5x.png 1.5x, /static/images/wikimedia-button-2x.png 2x" width="88"/>
     </a>
    </li>
    <li id="footer-poweredbyico">
     <a href="//www.mediawiki.org/">
      <img alt="Powered by MediaWiki" height="31" src="/static/images/poweredby_mediawiki_88x31.png" srcset="/static/images/poweredby_mediawiki_132x47.png 1.5x, /static/images/poweredby_mediawiki_176x62.png 2x" width="88"/>
     </a>
    </li>
   </ul>
   <div style="clear: both;"/>
  </div>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgPageParseReport":{"limitreport":{"cputime":"0.324","walltime":"0.395","ppvisitednodes":{"value":5142,"limit":1000000},"ppgeneratednodes":{"value":0,"limit":1500000},"postexpandincludesize":{"value":79976,"limit":2097152},"templateargumentsize":{"value":991,"limit":2097152},"expansiondepth":{"value":12,"limit":40},"expensivefunctioncount":{"value":2,"limit":500},"unstrip-depth":{"value":1,"limit":20},"unstrip-size":{"value":10101,"limit":5000000},"entityaccesscount":{"value":0,"limit":400},"timingprofile":["100.00%  279.718      1 -total"," 41.33%  115.594      2 Template:Reflist"," 32.47%   90.827      2 Template:London_Gazette"," 30.02%   83.982      2 Template:Cite_magazine"," 20.77%   58.105     33 Template:Coord","  9.59%   26.818     33 Template:English_district_control","  9.47%   26.499      1 Template:Use_dmy_dates","  5.78%   16.180      1 Template:London","  4.97%   13.913      2 Template:DMCA","  4.23%   11.829      2 Template:Dated_maintenance_category"]},"scribunto":{"limitreport-timeusage":{"value":"0.111","limit":"10.000"},"limitreport-memusage":{"value":2885183,"limit":52428800}},"cachereport":{"origin":"mw1269","timestamp":"20190428014242","ttl":2592000,"transientcontent":false}}});});
  </script>
  <script type="application/ld+json">
   {"@context":"https:\/\/schema.org","@type":"Article","name":"List of London boroughs","url":"https:\/\/en.wikipedia.org\/wiki\/List_of_London_boroughs","sameAs":"http:\/\/www.wikidata.org\/entity\/Q6577004","mainEntity":"http:\/\/www.wikidata.org\/entity\/Q6577004","author":{"@type":"Organization","name":"Contributors to Wikimedia projects"},"publisher":{"@type":"Organization","name":"Wikimedia Foundation, Inc.","logo":{"@type":"ImageObject","url":"https:\/\/www.wikimedia.org\/static\/images\/wmf-hor-googpub.png"}},"datePublished":"2010-07-20T07:28:35Z","dateModified":"2019-02-05T14:50:59Z","headline":"Wikimedia list article"}
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgBackendResponseTime":109,"wgHostname":"mw1274"});});
  </script>
 </body>
</html>

In [ ]:
# extracting the raw table inside that webpage
table = soup.find_all('table', {'class':'wikitable sortable'})
print(table)

[<table class="wikitable sortable" style="font-size:100%" width="100%">
<tbody><tr>
<th>Borough
</th>
<th>Inner
</th>
<th>Status
</th>
<th>Local authority
</th>
<th>Political control
</th>
<th>Headquarters
</th>
<th>Area (sq mi)
</th>
<th>Population (2013 est)<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup>
</th>
<th>Co-ordinates
</th>
<th><span style="background:#67BCD3"> Nr. in map </span>
</th></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Barking_and_Dagenham" title="London Borough of Barking and Dagenham">Barking and Dagenham</a> <sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[note 1]</a></sup>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Barking_and_Dagenham_London_Borough_Council" title="Barking and Dagenham London Borough Council">Barking and Dagenham London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Barking_Town_Hall&amp;action=edit&amp;redlink=1" title="Barking Town Hall (page does not exist)">Town Hall</a>, 1 Town Square
</td>
<td>13.93
</td>
<td>194,352
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5607_N_0.1557_E_region:GB_type:city&amp;title=Barking+and+Dagenham"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°33′39″N</span> <span class="longitude">0°09′21″E</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5607°N 0.1557°E</span><span style="display:none">﻿ / <span class="geo">51.5607; 0.1557</span></span><span style="display:none">﻿ (<span class="fn org">Barking and Dagenham</span>)</span></span></span></a></span>
</td>
<td>25
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Barnet" title="London Borough of Barnet">Barnet</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Barnet_London_Borough_Council" title="Barnet London Borough Council">Barnet London Borough Council</a>
</td>
<td><a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">Conservative</a>
</td>
<td><a class="new" href="/w/index.php?title=North_London_Business_Park&amp;action=edit&amp;redlink=1" title="North London Business Park (page does not exist)">North London Business Park</a>, Oakleigh Road South
</td>
<td>33.49
</td>
<td>369,088
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.6252_N_0.1517_W_region:GB_type:city&amp;title=Barnet"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°37′31″N</span> <span class="longitude">0°09′06″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.6252°N 0.1517°W</span><span style="display:none">﻿ / <span class="geo">51.6252; -0.1517</span></span><span style="display:none">﻿ (<span class="fn org">Barnet</span>)</span></span></span></a></span>
</td>
<td>31
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Bexley" title="London Borough of Bexley">Bexley</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Bexley_London_Borough_Council" title="Bexley London Borough Council">Bexley London Borough Council</a>
</td>
<td><a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">Conservative</a>
</td>
<td><a class="new" href="/w/index.php?title=Civic_Offices&amp;action=edit&amp;redlink=1" title="Civic Offices (page does not exist)">Civic Offices</a>, 2 Watling Street
</td>
<td>23.38
</td>
<td>236,687
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4549_N_0.1505_E_region:GB_type:city&amp;title=Bexley"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°27′18″N</span> <span class="longitude">0°09′02″E</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.4549°N 0.1505°E</span><span style="display:none">﻿ / <span class="geo">51.4549; 0.1505</span></span><span style="display:none">﻿ (<span class="fn org">Bexley</span>)</span></span></span></a></span>
</td>
<td>23
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Brent" title="London Borough of Brent">Brent</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Brent_London_Borough_Council" title="Brent London Borough Council">Brent London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a href="/wiki/Brent_Civic_Centre" title="Brent Civic Centre">Brent Civic Centre</a>, Engineers Way
</td>
<td>16.70
</td>
<td>317,264
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5588_N_0.2817_W_region:GB_type:city&amp;title=Brent"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°33′32″N</span> <span class="longitude">0°16′54″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5588°N 0.2817°W</span><span style="display:none">﻿ / <span class="geo">51.5588; -0.2817</span></span><span style="display:none">﻿ (<span class="fn org">Brent</span>)</span></span></span></a></span>
</td>
<td>12
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Bromley" title="London Borough of Bromley">Bromley</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Bromley_London_Borough_Council" title="Bromley London Borough Council">Bromley London Borough Council</a>
</td>
<td><a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">Conservative</a>
</td>
<td><a class="new" href="/w/index.php?title=Bromley_Civic_Centre&amp;action=edit&amp;redlink=1" title="Bromley Civic Centre (page does not exist)">Civic Centre</a>, Stockwell Close
</td>
<td>57.97
</td>
<td>317,899
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4039_N_0.0198_E_region:GB_type:city&amp;title=Bromley"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°24′14″N</span> <span class="longitude">0°01′11″E</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.4039°N 0.0198°E</span><span style="display:none">﻿ / <span class="geo">51.4039; 0.0198</span></span><span style="display:none">﻿ (<span class="fn org">Bromley</span>)</span></span></span></a></span>
</td>
<td>20
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Camden" title="London Borough of Camden">Camden</a>
</td>
<td><img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/><span style="display:none">Y</span>
</td>
<td>
</td>
<td><a href="/wiki/Camden_London_Borough_Council" title="Camden London Borough Council">Camden London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a href="/wiki/Camden_Town_Hall" title="Camden Town Hall">Camden Town Hall</a>, Judd Street
</td>
<td>8.40
</td>
<td>229,719
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.529_N_0.1255_W_region:GB_type:city&amp;title=Camden"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°31′44″N</span> <span class="longitude">0°07′32″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5290°N 0.1255°W</span><span style="display:none">﻿ / <span class="geo">51.5290; -0.1255</span></span><span style="display:none">﻿ (<span class="fn org">Camden</span>)</span></span></span></a></span>
</td>
<td>11
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Croydon" title="London Borough of Croydon">Croydon</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Croydon_London_Borough_Council" title="Croydon London Borough Council">Croydon London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Bernard_Weatherill_House&amp;action=edit&amp;redlink=1" title="Bernard Weatherill House (page does not exist)">Bernard Weatherill House</a>, Mint Walk
</td>
<td>33.41
</td>
<td>372,752
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.3714_N_0.0977_W_region:GB_type:city&amp;title=Croydon"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°22′17″N</span> <span class="longitude">0°05′52″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.3714°N 0.0977°W</span><span style="display:none">﻿ / <span class="geo">51.3714; -0.0977</span></span><span style="display:none">﻿ (<span class="fn org">Croydon</span>)</span></span></span></a></span>
</td>
<td>19
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Ealing" title="London Borough of Ealing">Ealing</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Ealing_London_Borough_Council" title="Ealing London Borough Council">Ealing London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Perceval_House,_Ealing&amp;action=edit&amp;redlink=1" title="Perceval House, Ealing (page does not exist)">Perceval House</a>, 14-16 Uxbridge Road
</td>
<td>21.44
</td>
<td>342,494
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.513_N_0.3089_W_region:GB_type:city&amp;title=Ealing"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°30′47″N</span> <span class="longitude">0°18′32″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5130°N 0.3089°W</span><span style="display:none">﻿ / <span class="geo">51.5130; -0.3089</span></span><span style="display:none">﻿ (<span class="fn org">Ealing</span>)</span></span></span></a></span>
</td>
<td>13
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Enfield" title="London Borough of Enfield">Enfield</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Enfield_London_Borough_Council" title="Enfield London Borough Council">Enfield London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Enfield_Civic_Centre&amp;action=edit&amp;redlink=1" title="Enfield Civic Centre (page does not exist)">Civic Centre</a>, Silver Street
</td>
<td>31.74
</td>
<td>320,524
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.6538_N_0.0799_W_region:GB_type:city&amp;title=Enfield"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°39′14″N</span> <span class="longitude">0°04′48″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.6538°N 0.0799°W</span><span style="display:none">﻿ / <span class="geo">51.6538; -0.0799</span></span><span style="display:none">﻿ (<span class="fn org">Enfield</span>)</span></span></span></a></span>
</td>
<td>30
</td></tr>
<tr>
<td><a href="/wiki/Royal_Borough_of_Greenwich" title="Royal Borough of Greenwich">Greenwich</a> <sup class="reference" id="cite_ref-3"><a href="#cite_note-3">[note 2]</a></sup>
</td>
<td><img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/><span style="display:none">Y</span> <sup class="reference" id="cite_ref-note2_4-0"><a href="#cite_note-note2-4">[note 3]</a></sup>
</td>
<td><a class="mw-redirect" href="/wiki/Royal_borough" title="Royal borough">Royal</a>
</td>
<td><a href="/wiki/Greenwich_London_Borough_Council" title="Greenwich London Borough Council">Greenwich London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a href="/wiki/Woolwich_Town_Hall" title="Woolwich Town Hall">Woolwich Town Hall</a>, Wellington Street
</td>
<td>18.28
</td>
<td>264,008
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4892_N_0.0648_E_region:GB_type:city&amp;title=Greenwich"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°29′21″N</span> <span class="longitude">0°03′53″E</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.4892°N 0.0648°E</span><span style="display:none">﻿ / <span class="geo">51.4892; 0.0648</span></span><span style="display:none">﻿ (<span class="fn org">Greenwich</span>)</span></span></span></a></span>
</td>
<td>22
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Hackney" title="London Borough of Hackney">Hackney</a>
</td>
<td><img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/><span style="display:none">Y</span>
</td>
<td>
</td>
<td><a href="/wiki/Hackney_London_Borough_Council" title="Hackney London Borough Council">Hackney London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Hackney_Town_Hall&amp;action=edit&amp;redlink=1" title="Hackney Town Hall (page does not exist)">Hackney Town Hall</a>, Mare Street
</td>
<td>7.36
</td>
<td>257,379
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.545_N_0.0553_W_region:GB_type:city&amp;title=Hackney"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°32′42″N</span> <span class="longitude">0°03′19″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5450°N 0.0553°W</span><span style="display:none">﻿ / <span class="geo">51.5450; -0.0553</span></span><span style="display:none">﻿ (<span class="fn org">Hackney</span>)</span></span></span></a></span>
</td>
<td>9
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Hammersmith_and_Fulham" title="London Borough of Hammersmith and Fulham">Hammersmith and Fulham</a> <sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[note 4]</a></sup>
</td>
<td><img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/><span style="display:none">Y</span>
</td>
<td>
</td>
<td><a href="/wiki/Hammersmith_and_Fulham_London_Borough_Council" title="Hammersmith and Fulham London Borough Council">Hammersmith and Fulham London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Hammersmith_and_Fulham_Town_Hall&amp;action=edit&amp;redlink=1" title="Hammersmith and Fulham Town Hall (page does not exist)">Town Hall</a>, King Street
</td>
<td>6.33
</td>
<td>178,685
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4927_N_0.2339_W_region:GB_type:city&amp;title=Hammersmith+and+Fulham"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°29′34″N</span> <span class="longitude">0°14′02″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.4927°N 0.2339°W</span><span style="display:none">﻿ / <span class="geo">51.4927; -0.2339</span></span><span style="display:none">﻿ (<span class="fn org">Hammersmith and Fulham</span>)</span></span></span></a></span>
</td>
<td>4
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Haringey" title="London Borough of Haringey">Haringey</a>
</td>
<td><sup class="reference" id="cite_ref-note2_4-1"><a href="#cite_note-note2-4">[note 3]</a></sup>
</td>
<td>
</td>
<td><a href="/wiki/Haringey_London_Borough_Council" title="Haringey London Borough Council">Haringey London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Haringey_Civic_Centre&amp;action=edit&amp;redlink=1" title="Haringey Civic Centre (page does not exist)">Civic Centre</a>, High Road
</td>
<td>11.42
</td>
<td>263,386
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.6_N_0.1119_W_region:GB_type:city&amp;title=Haringey"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°36′00″N</span> <span class="longitude">0°06′43″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.6000°N 0.1119°W</span><span style="display:none">﻿ / <span class="geo">51.6000; -0.1119</span></span><span style="display:none">﻿ (<span class="fn org">Haringey</span>)</span></span></span></a></span>
</td>
<td>29
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Harrow" title="London Borough of Harrow">Harrow</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Harrow_London_Borough_Council" title="Harrow London Borough Council">Harrow London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Harrow_Civic_Centre&amp;action=edit&amp;redlink=1" title="Harrow Civic Centre (page does not exist)">Civic Centre</a>, Station Road
</td>
<td>19.49
</td>
<td>243,372
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5898_N_0.3346_W_region:GB_type:city&amp;title=Harrow"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°35′23″N</span> <span class="longitude">0°20′05″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5898°N 0.3346°W</span><span style="display:none">﻿ / <span class="geo">51.5898; -0.3346</span></span><span style="display:none">﻿ (<span class="fn org">Harrow</span>)</span></span></span></a></span>
</td>
<td>32
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Havering" title="London Borough of Havering">Havering</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Havering_London_Borough_Council" title="Havering London Borough Council">Havering London Borough Council</a>
</td>
<td><a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">Conservative</a> (council <a href="/wiki/No_overall_control" title="No overall control">NOC</a>)
</td>
<td><a class="new" href="/w/index.php?title=Havering_Town_Hall&amp;action=edit&amp;redlink=1" title="Havering Town Hall (page does not exist)">Town Hall</a>, Main Road
</td>
<td>43.35
</td>
<td>242,080
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5812_N_0.1837_E_region:GB_type:city&amp;title=Havering"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°34′52″N</span> <span class="longitude">0°11′01″E</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5812°N 0.1837°E</span><span style="display:none">﻿ / <span class="geo">51.5812; 0.1837</span></span><span style="display:none">﻿ (<span class="fn org">Havering</span>)</span></span></span></a></span>
</td>
<td>24
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Hillingdon" title="London Borough of Hillingdon">Hillingdon</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Hillingdon_London_Borough_Council" title="Hillingdon London Borough Council">Hillingdon London Borough Council</a>
</td>
<td><a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">Conservative</a>
</td>
<td><a href="/wiki/Hillingdon_Civic_Centre" title="Hillingdon Civic Centre">Civic Centre</a>, High Street
</td>
<td>44.67
</td>
<td>286,806
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5441_N_0.476_W_region:GB_type:city&amp;title=Hillingdon"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°32′39″N</span> <span class="longitude">0°28′34″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5441°N 0.4760°W</span><span style="display:none">﻿ / <span class="geo">51.5441; -0.4760</span></span><span style="display:none">﻿ (<span class="fn org">Hillingdon</span>)</span></span></span></a></span>
</td>
<td>33
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Hounslow" title="London Borough of Hounslow">Hounslow</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Hounslow_London_Borough_Council" title="Hounslow London Borough Council">Hounslow London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Hounslow_Civic_Centre&amp;action=edit&amp;redlink=1" title="Hounslow Civic Centre (page does not exist)">Civic Centre</a>, Lampton Road
</td>
<td>21.61
</td>
<td>262,407
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4746_N_0.368_W_region:GB_type:city&amp;title=Hounslow"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°28′29″N</span> <span class="longitude">0°22′05″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.4746°N 0.3680°W</span><span style="display:none">﻿ / <span class="geo">51.4746; -0.3680</span></span><span style="display:none">﻿ (<span class="fn org">Hounslow</span>)</span></span></span></a></span>
</td>
<td>14
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Islington" title="London Borough of Islington">Islington</a>
</td>
<td><img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/><span style="display:none">Y</span>
</td>
<td>
</td>
<td><a href="/wiki/Islington_London_Borough_Council" title="Islington London Borough Council">Islington London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Islington_Municipal_Offices&amp;action=edit&amp;redlink=1" title="Islington Municipal Offices (page does not exist)">Municipal Offices</a>, 222 Upper Street
</td>
<td>5.74
</td>
<td>215,667
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5416_N_0.1022_W_region:GB_type:city&amp;title=Islington"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°32′30″N</span> <span class="longitude">0°06′08″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5416°N 0.1022°W</span><span style="display:none">﻿ / <span class="geo">51.5416; -0.1022</span></span><span style="display:none">﻿ (<span class="fn org">Islington</span>)</span></span></span></a></span>
</td>
<td>10
</td></tr>
<tr>
<td><a href="/wiki/Royal_Borough_of_Kensington_and_Chelsea" title="Royal Borough of Kensington and Chelsea">Kensington and Chelsea</a>
</td>
<td><img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/><span style="display:none">Y</span>
</td>
<td><a class="mw-redirect" href="/wiki/Royal_borough" title="Royal borough">Royal</a>
</td>
<td><a href="/wiki/Kensington_and_Chelsea_London_Borough_Council" title="Kensington and Chelsea London Borough Council">Kensington and Chelsea London Borough Council</a>
</td>
<td><a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">Conservative</a>
</td>
<td><a href="/wiki/Kensington_Town_Hall,_London" title="Kensington Town Hall, London">The Town Hall</a>, <a href="/wiki/Hornton_Street" title="Hornton Street">Hornton Street</a>
</td>
<td>4.68
</td>
<td>155,594
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.502_N_0.1947_W_region:GB_type:city&amp;title=Kensington+and+Chelsea"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°30′07″N</span> <span class="longitude">0°11′41″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5020°N 0.1947°W</span><span style="display:none">﻿ / <span class="geo">51.5020; -0.1947</span></span><span style="display:none">﻿ (<span class="fn org">Kensington and Chelsea</span>)</span></span></span></a></span>
</td>
<td>3
</td></tr>
<tr>
<td><a href="/wiki/Royal_Borough_of_Kingston_upon_Thames" title="Royal Borough of Kingston upon Thames">Kingston upon Thames</a>
</td>
<td>
</td>
<td><a class="mw-redirect" href="/wiki/Royal_borough" title="Royal borough">Royal</a>
</td>
<td><a href="/wiki/Kingston_upon_Thames_London_Borough_Council" title="Kingston upon Thames London Borough Council">Kingston upon Thames London Borough Council</a>
</td>
<td><a href="/wiki/Liberal_Democrats_(UK)" title="Liberal Democrats (UK)">Liberal Democrat</a>
</td>
<td><a href="/wiki/Kingston_upon_Thames_Guildhall" title="Kingston upon Thames Guildhall">Guildhall</a>, High Street
</td>
<td>14.38
</td>
<td>166,793
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4085_N_0.3064_W_region:GB_type:city&amp;title=Kingston+upon+Thames"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°24′31″N</span> <span class="longitude">0°18′23″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.4085°N 0.3064°W</span><span style="display:none">﻿ / <span class="geo">51.4085; -0.3064</span></span><span style="display:none">﻿ (<span class="fn org">Kingston upon Thames</span>)</span></span></span></a></span>
</td>
<td>16
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Lambeth" title="London Borough of Lambeth">Lambeth</a>
</td>
<td><img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/><span style="display:none">Y</span>
</td>
<td>
</td>
<td><a href="/wiki/Lambeth_London_Borough_Council" title="Lambeth London Borough Council">Lambeth London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a href="/wiki/Lambeth_Town_Hall" title="Lambeth Town Hall">Lambeth Town Hall</a>, Brixton Hill
</td>
<td>10.36
</td>
<td>314,242
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4607_N_0.1163_W_region:GB_type:city&amp;title=Lambeth"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°27′39″N</span> <span class="longitude">0°06′59″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.4607°N 0.1163°W</span><span style="display:none">﻿ / <span class="geo">51.4607; -0.1163</span></span><span style="display:none">﻿ (<span class="fn org">Lambeth</span>)</span></span></span></a></span>
</td>
<td>6
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Lewisham" title="London Borough of Lewisham">Lewisham</a>
</td>
<td><img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/><span style="display:none">Y</span>
</td>
<td>
</td>
<td><a href="/wiki/Lewisham_London_Borough_Council" title="Lewisham London Borough Council">Lewisham London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Lewisham_Town_Hall&amp;action=edit&amp;redlink=1" title="Lewisham Town Hall (page does not exist)">Town Hall</a>, 1 Catford Road
</td>
<td>13.57
</td>
<td>286,180
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4452_N_0.0209_W_region:GB_type:city&amp;title=Lewisham"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°26′43″N</span> <span class="longitude">0°01′15″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.4452°N 0.0209°W</span><span style="display:none">﻿ / <span class="geo">51.4452; -0.0209</span></span><span style="display:none">﻿ (<span class="fn org">Lewisham</span>)</span></span></span></a></span>
</td>
<td>21
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Merton" title="London Borough of Merton">Merton</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Merton_London_Borough_Council" title="Merton London Borough Council">Merton London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Merton_Civic_Centre&amp;action=edit&amp;redlink=1" title="Merton Civic Centre (page does not exist)">Civic Centre</a>, London Road
</td>
<td>14.52
</td>
<td>203,223
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4014_N_0.1958_W_region:GB_type:city&amp;title=Merton"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°24′05″N</span> <span class="longitude">0°11′45″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.4014°N 0.1958°W</span><span style="display:none">﻿ / <span class="geo">51.4014; -0.1958</span></span><span style="display:none">﻿ (<span class="fn org">Merton</span>)</span></span></span></a></span>
</td>
<td>17
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Newham" title="London Borough of Newham">Newham</a>
</td>
<td><sup class="reference" id="cite_ref-note2_4-2"><a href="#cite_note-note2-4">[note 3]</a></sup>
</td>
<td>
</td>
<td><a href="/wiki/Newham_London_Borough_Council" title="Newham London Borough Council">Newham London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Newham_Dockside&amp;action=edit&amp;redlink=1" title="Newham Dockside (page does not exist)">Newham Dockside</a>, 1000 Dockside Road
</td>
<td>13.98
</td>
<td>318,227
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5077_N_0.0469_E_region:GB_type:city&amp;title=Newham"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°30′28″N</span> <span class="longitude">0°02′49″E</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5077°N 0.0469°E</span><span style="display:none">﻿ / <span class="geo">51.5077; 0.0469</span></span><span style="display:none">﻿ (<span class="fn org">Newham</span>)</span></span></span></a></span>
</td>
<td>27
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Redbridge" title="London Borough of Redbridge">Redbridge</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Redbridge_London_Borough_Council" title="Redbridge London Borough Council">Redbridge London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Redbridge_Town_Hall&amp;action=edit&amp;redlink=1" title="Redbridge Town Hall (page does not exist)">Town Hall</a>, 128-142 High Road
</td>
<td>21.78
</td>
<td>288,272
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.559_N_0.0741_E_region:GB_type:city&amp;title=Redbridge"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°33′32″N</span> <span class="longitude">0°04′27″E</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5590°N 0.0741°E</span><span style="display:none">﻿ / <span class="geo">51.5590; 0.0741</span></span><span style="display:none">﻿ (<span class="fn org">Redbridge</span>)</span></span></span></a></span>
</td>
<td>26
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Richmond_upon_Thames" title="London Borough of Richmond upon Thames">Richmond upon Thames</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Richmond_upon_Thames_London_Borough_Council" title="Richmond upon Thames London Borough Council">Richmond upon Thames London Borough Council</a>
</td>
<td><a href="/wiki/Liberal_Democrats_(UK)" title="Liberal Democrats (UK)">Liberal Democrat</a>
</td>
<td><a class="new" href="/w/index.php?title=Richmond_upon_Thames_Civic_Centre&amp;action=edit&amp;redlink=1" title="Richmond upon Thames Civic Centre (page does not exist)">Civic Centre</a>, 44 York Street
</td>
<td>22.17
</td>
<td>191,365
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4479_N_0.326_W_region:GB_type:city&amp;title=Richmond+upon+Thames"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°26′52″N</span> <span class="longitude">0°19′34″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.4479°N 0.3260°W</span><span style="display:none">﻿ / <span class="geo">51.4479; -0.3260</span></span><span style="display:none">﻿ (<span class="fn org">Richmond upon Thames</span>)</span></span></span></a></span>
</td>
<td>15
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Southwark" title="London Borough of Southwark">Southwark</a>
</td>
<td><img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/><span style="display:none">Y</span>
</td>
<td>
</td>
<td><a href="/wiki/Southwark_London_Borough_Council" title="Southwark London Borough Council">Southwark London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=160_Tooley_Street&amp;action=edit&amp;redlink=1" title="160 Tooley Street (page does not exist)">160 Tooley Street</a>
</td>
<td>11.14
</td>
<td>298,464
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5035_N_0.0804_W_region:GB_type:city&amp;title=Southwark"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°30′13″N</span> <span class="longitude">0°04′49″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5035°N 0.0804°W</span><span style="display:none">﻿ / <span class="geo">51.5035; -0.0804</span></span><span style="display:none">﻿ (<span class="fn org">Southwark</span>)</span></span></span></a></span>
</td>
<td>7
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Sutton" title="London Borough of Sutton">Sutton</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Sutton_London_Borough_Council" title="Sutton London Borough Council">Sutton London Borough Council</a>
</td>
<td><a href="/wiki/Liberal_Democrats_(UK)" title="Liberal Democrats (UK)">Liberal Democrat</a>
</td>
<td><a class="new" href="/w/index.php?title=Sutton_Civic_Offices&amp;action=edit&amp;redlink=1" title="Sutton Civic Offices (page does not exist)">Civic Offices</a>, St Nicholas Way
</td>
<td>16.93
</td>
<td>195,914
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.3618_N_0.1945_W_region:GB_type:city&amp;title=Sutton"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°21′42″N</span> <span class="longitude">0°11′40″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.3618°N 0.1945°W</span><span style="display:none">﻿ / <span class="geo">51.3618; -0.1945</span></span><span style="display:none">﻿ (<span class="fn org">Sutton</span>)</span></span></span></a></span>
</td>
<td>18
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Tower_Hamlets" title="London Borough of Tower Hamlets">Tower Hamlets</a>
</td>
<td><img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/><span style="display:none">Y</span>
</td>
<td>
</td>
<td><a href="/wiki/Tower_Hamlets_London_Borough_Council" title="Tower Hamlets London Borough Council">Tower Hamlets London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a class="new" href="/w/index.php?title=Tower_Hamlets_Town_Hall&amp;action=edit&amp;redlink=1" title="Tower Hamlets Town Hall (page does not exist)">Town Hall</a>, Mulberry Place, 5 Clove Crescent
</td>
<td>7.63
</td>
<td>272,890
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5099_N_0.0059_W_region:GB_type:city&amp;title=Tower+Hamlets"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°30′36″N</span> <span class="longitude">0°00′21″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5099°N 0.0059°W</span><span style="display:none">﻿ / <span class="geo">51.5099; -0.0059</span></span><span style="display:none">﻿ (<span class="fn org">Tower Hamlets</span>)</span></span></span></a></span>
</td>
<td>8
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Waltham_Forest" title="London Borough of Waltham Forest">Waltham Forest</a>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Waltham_Forest_London_Borough_Council" title="Waltham Forest London Borough Council">Waltham Forest London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a href="/wiki/Waltham_Forest_Town_Hall" title="Waltham Forest Town Hall">Waltham Forest Town Hall</a>, Forest Road
</td>
<td>14.99
</td>
<td>265,797
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5908_N_0.0134_W_region:GB_type:city&amp;title=Waltham+Forest"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°35′27″N</span> <span class="longitude">0°00′48″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5908°N 0.0134°W</span><span style="display:none">﻿ / <span class="geo">51.5908; -0.0134</span></span><span style="display:none">﻿ (<span class="fn org">Waltham Forest</span>)</span></span></span></a></span>
</td>
<td>28
</td></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Wandsworth" title="London Borough of Wandsworth">Wandsworth</a>
</td>
<td><img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/><span style="display:none">Y</span>
</td>
<td>
</td>
<td><a href="/wiki/Wandsworth_London_Borough_Council" title="Wandsworth London Borough Council">Wandsworth London Borough Council</a>
</td>
<td><a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">Conservative</a>
</td>
<td><a class="new" href="/w/index.php?title=Wandsworth_Town_Hall&amp;action=edit&amp;redlink=1" title="Wandsworth Town Hall (page does not exist)">The Town Hall</a>, <a href="/wiki/Wandsworth_High_Street" title="Wandsworth High Street">Wandsworth High Street</a>
</td>
<td>13.23
</td>
<td>310,516
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4567_N_0.191_W_region:GB_type:city&amp;title=Wandsworth"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°27′24″N</span> <span class="longitude">0°11′28″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.4567°N 0.1910°W</span><span style="display:none">﻿ / <span class="geo">51.4567; -0.1910</span></span><span style="display:none">﻿ (<span class="fn org">Wandsworth</span>)</span></span></span></a></span>
</td>
<td>5
</td></tr>
<tr>
<td><a href="/wiki/City_of_Westminster" title="City of Westminster">Westminster</a>
</td>
<td><img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/><span style="display:none">Y</span>
</td>
<td><a href="/wiki/City_status_in_the_United_Kingdom" title="City status in the United Kingdom">City</a>
</td>
<td><a href="/wiki/Westminster_City_Council" title="Westminster City Council">Westminster City Council</a>
</td>
<td><a href="/wiki/Conservative_Party_(UK)" title="Conservative Party (UK)">Conservative</a>
</td>
<td><a class="new" href="/w/index.php?title=Westminster_City_Hall&amp;action=edit&amp;redlink=1" title="Westminster City Hall (page does not exist)">Westminster City Hall</a>, 64 Victoria Street
</td>
<td>8.29
</td>
<td>226,841
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.4973_N_0.1372_W_region:GB_type:city&amp;title=Westminster"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°29′50″N</span> <span class="longitude">0°08′14″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.4973°N 0.1372°W</span><span style="display:none">﻿ / <span class="geo">51.4973; -0.1372</span></span><span style="display:none">﻿ (<span class="fn org">Westminster</span>)</span></span></span></a></span>
</td>
<td>2
</td></tr></tbody></table>, <table class="wikitable sortable" style="font-size:95%" width="100%">
<tbody><tr>
<th width="100px"><i>‘Borough’</i>
</th>
<th>Inner
</th>
<th width="100px">Status
</th>
<th>Local authority
</th>
<th>Political control
</th>
<th width="120px">Headquarters
</th>
<th>Area<br/>(sq mi)
</th>
<th>Population<br/>(2011 est)
</th>
<th width="20px">Co-ordinates
</th>
<th><span style="background:#67BCD3"> Nr. in<br/>map </span>
</th></tr>
<tr>
<td><a href="/wiki/City_of_London" title="City of London">City of London</a>
</td>
<td>(<img alt="☑" data-file-height="600" data-file-width="600" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/20px-Yes_check.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/30px-Yes_check.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/fb/Yes_check.svg/40px-Yes_check.svg.png 2x" width="20"/><span style="display:none">Y</span>)<br/><sup class="reference" id="cite_ref-6"><a href="#cite_note-6">[note 5]</a></sup>
</td>
<td><i><a href="/wiki/Sui_generis" title="Sui generis">Sui generis</a></i>;<br/><a href="/wiki/City_status_in_the_United_Kingdom" title="City status in the United Kingdom">City</a>;<br/><a href="/wiki/Ceremonial_counties_of_England" title="Ceremonial counties of England">Ceremonial county</a>
</td>
<td><a class="mw-redirect" href="/wiki/Corporation_of_London" title="Corporation of London">Corporation of London</a>;<br/><a href="/wiki/Inner_Temple" title="Inner Temple">Inner Temple</a>;<br/><a href="/wiki/Middle_Temple" title="Middle Temple">Middle Temple</a>
</td>
<td>? 
</td>
<td><a href="/wiki/Guildhall,_London" title="Guildhall, London">Guildhall</a>
</td>
<td>1.12
</td>
<td>7,000
</td>
<td><span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?pagename=List_of_London_boroughs&amp;params=51.5155_N_0.0922_W_region:GB_type:city&amp;title=City+of+London"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">51°30′56″N</span> <span class="longitude">0°05′32″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">51.5155°N 0.0922°W</span><span style="display:none">﻿ / <span class="geo">51.5155; -0.0922</span></span><span style="display:none">﻿ (<span class="fn org">City of London</span>)</span></span></span></a></span>
</td>
<td>1
</td></tr></tbody></table>]

### Converting the table into a data frame

In [ ]:
London_table = pd.read_html(str(table[0]), index_col=None, header=0)[0]
London_table.head()

Borough    Inner   Status   Local       Political   Headquarters  population
                            authority     control  
    
Barking    NaN     NaN     council      Labour       town sqaure     655686
and Dagam
    
Barnet     NaN     NaN      council    Conservative   North london   765676
    
Bexley     NaN     NaN      council     Labour       walting street  465465
    
Brent      NaN     NaN      council    Conservative   Brent center    56546
    
Bromley    NaN     NaN      council      Labour         stockwell    345345
    
    

#### The second table on the site contains the addition Borough i.e. City of London

In [ ]:
# Read in the second table 
London_table1 = pd.read_html(str(table[1]), index_col=None, header=0)[0]

# Rename the columns to match the previous table to append the tables.

London_table1.columns = ['Borough','Inner','Status','Local authority','Political control',
                         'Headquarters','Area (sq mi)','Population (2013 est)[1]','Co-ordinates','Nr. in map']

# View the table
London_table1

Borough    Inner   Status   Local       Political   Headquarters  population
                            authority     control  
    
Barking    NaN     NaN     council      Labour       town sqaure    6545686
and Dagam

#### Append the data frame together

In [ ]:
# A continuous index value will be maintained 
# across the rows in the new appended data frame. 

London_table = London_table.append(London_table1, ignore_index = True) 
London_table.head()

Borough    Inner   Status   Local       Political   Headquarters  population
                            authority     control  
    
Barking    NaN     NaN     council      Labour       town sqaure    6545686
and Dagam
    
Barnet     NaN     NaN      council    Conservative   North london   765676
    
Bexley     NaN     NaN      council     Labour       walting street  465465
    
Brent      NaN     NaN      council    Conservative   Brent center    56546
    
Bromley    NaN     NaN      council      Labour         stockwell    345345

#### Check if the last row was appended correctly

In [ ]:
London_table.tail()

Borough    Inner   Status   Local       Political   Headquarters  population
                            authority     control  
    
Barking    NaN     NaN     council      Labour       town sqaure    6545686
and Dagam
    
Barnet     NaN     NaN      council    Conservative   North london   765676

### View the information of the data set

In [ ]:
London_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 10 columns):
Borough                     33 non-null object
Inner                       15 non-null object
Status                      5 non-null object
Local authority             33 non-null object
Political control           33 non-null object
Headquarters                33 non-null object
Area (sq mi)                33 non-null float64
Population (2013 est)[1]    33 non-null int64
Co-ordinates                33 non-null object
Nr. in map                  33 non-null int64
dtypes: float64(1), int64(2), object(7)
memory usage: 2.7+ KB

### Removing Unnecessary string in the Data set

In [ ]:
London_table = London_table.replace('note 1','', regex=True) 
London_table = London_table.replace('note 2','', regex=True) 
London_table = London_table.replace('note 3','', regex=True) 
London_table = London_table.replace('note 4','', regex=True) 
London_table = London_table.replace('note 5','', regex=True) 

# View the top of the data set
London_table.head()

In [ ]:
Borough    Inner   Status   Local       Political   Headquarters  population
                            authority     control  
    
Barking    NaN     NaN     council      Labour       town sqaure    6545686
and Dagam
    
Barnet     NaN     NaN      council    Conservative   North london   765676
    
Bexley     NaN     NaN      council     Labour       walting street  465465
    
Brent      NaN     NaN      council    Conservative   Brent center    56546
    
Bromley    NaN     NaN      council      Labour         stockwell    345345

#### Check the type of the newly created table

In [ ]:
type(London_table)

pandas.core.frame.DataFrame

In [ ]:
# Shape of the data frame
London_table.shape

(33, 10)

### Check if the Borough in both the data frames match.

In [ ]:
set(df.Borough) - set(London_table.Borough)

{'Barking and Dagenham', 'Greenwich', 'Hammersmith and Fulham'}
These 3 Boroughs don't match because of the unnecessary symobols present "[]"

### Find the index of the Boroughs that didn't match

In [ ]:
print("The index of first borough is",London_table.index[London_table['Borough'] == 'Barking and Dagenham []'].tolist())
print("The index of second borough is",London_table.index[London_table['Borough'] == 'Greenwich []'].tolist())
print("The index of third borough is",London_table.index[London_table['Borough'] == 'Hammersmith and Fulham []'].tolist())

The index of first borough is [0]
The index of second borough is [9]
The index of third borough is [11]

### Changing the Borough names to match the other data frame

In [ ]:
London_table.iloc[0,0] = 'Barking and Dagenham'
London_table.iloc[9,0] = 'Greenwich'
London_table.iloc[11,0] = 'Hammersmith and Fulham'

#### Check if the Borough names in both data sets match

In [ ]:
set(df.Borough) - set(London_table.Borough)

set()
The Borough names in both data frames match

### We can combine both the data frames together

In [ ]:
Ld_crime = pd.merge(London_crime, London_table, on='Borough')
Ld_crime.head(10)

Borough    Inner   Status   Local       Political   Headquarters  population
                            authority     control  
    
Barking    NaN     NaN     council      Labour       town sqaure    6545686
and Dagam
    
Barnet     NaN     NaN      council    Conservative   North london   765676
    
Bexley     NaN     NaN      council     Labour       walting street  465465
    
Brent      NaN     NaN      council    Conservative   Brent center    56546
    
Bromley    NaN     NaN      council      Labour         stockwell    345345

    
Barking    NaN     NaN     council      Labour       town sqaure    6545686
and Dagam
    
Barnet     NaN     NaN      council    Conservative   North london   765676
    
Bexley     NaN     NaN      council     Labour       walting street  465465
    
Brent      NaN     NaN      council    Conservative   Brent center    56546
    
Bromley    NaN     NaN      council      Labour         stockwell    345345
    
Barking    NaN     NaN     council      Labour       town sqaure    6545686
and Dagam
    
Barnet     NaN     NaN      council    Conservative   North london   765676
    
Bexley     NaN     NaN      council     Labour       walting street  465465
    
Brent      NaN     NaN      council    Conservative   Brent center    56546
    
Bromley    NaN     NaN      council      Labour         stockwell    345345
                           
    
Barking    NaN     NaN     council      Labour       town sqaure    6545686
and Dagam
    
Barnet     NaN     NaN      council    Conservative   North london   765676
    
Bexley     NaN     NaN      council     Labour       walting street  465465
    
Brent      NaN     NaN      council    Conservative   Brent center    56546
    
Bromley    NaN     NaN      council      Labour         stockwell    345345                           
    
Barking    NaN     NaN     council      Labour       town sqaure    6545686
and Dagam
    
Barnet     NaN     NaN      council    Conservative   North london   765676
    
Bexley     NaN     NaN      council     Labour       walting street  465465
    
Brent      NaN     NaN      council    Conservative   Brent center    56546
    
Bromley    NaN     NaN      council      Labour         stockwell    345345

In [ ]:
Ld_crime.shape

(33, 18)

In [ ]:
set(df.Borough) - set(Ld_crime.Borough)

set()

##### Rearranging the Columns

In [ ]:
# List of Column names of the data frame 
list(Ld_crime)

['Borough',
 'Burglary',
 'Criminal Damage',
 'Drugs',
 'Other Notifiable Offences',
 'Robbery',
 'Theft and Handling',
 'Violence Against the Person',
 'Total',
 'Inner',
 'Status',
 'Local authority',
 'Political control',
 'Headquarters',
 'Area (sq mi)',
 'Population (2013 est)[1]',
 'Co-ordinates',
 'Nr. in map']

In [ ]:
columnsTitles = ['Borough','Local authority','Political control','Headquarters',
                 'Area (sq mi)','Population (2013 est)[1]',
                 'Inner','Status',
                 'Burglary','Criminal Damage','Drugs','Other Notifiable Offences',
                 'Robbery','Theft and Handling','Violence Against the Person','Total','Co-ordinates']

Ld_crime = Ld_crime.reindex(columns=columnsTitles)

Ld_crime = Ld_crime[['Borough','Local authority','Political control','Headquarters',
                 'Area (sq mi)','Population (2013 est)[1]','Co-ordinates',
                 'Burglary','Criminal Damage','Drugs','Other Notifiable Offences',
                 'Robbery','Theft and Handling','Violence Against the Person','Total']]

Ld_crime.head()

Borough    Inner   Status   Local       Political   Headquarters  population
                            authority     control  
    
Barking    NaN     NaN     council      Labour       town sqaure    6545686
and Dagam
    
Barnet     NaN     NaN      council    Conservative   North london   765676
    
Bexley     NaN     NaN      council     Labour       walting street  465465
    
Brent      NaN     NaN      council    Conservative   Brent center    56546
    
Bromley    NaN     NaN      council      Labour         stockwell    345345

# Methodology 
The methodology in this project consists of two parts:

Exploratory Data Analysis: Visualise the crime rates in the London boroughs to idenity the safest borough and extract the neighborhoods in that borough to find the 10 most common venues in each neighborhood.
Modelling: To help people find similar neighborhoods in the safest borough we will be clustering similar neighborhoods using K - means clustering which is a form of unsupervised machine learning algorithm that clusters data based on predefined cluster size. We will use a cluster size of 5 for this project that will cluster the 15 neighborhoods into 5 clusters. The reason to conduct a K- means clustering is to cluster neighborhoods with similar venues together so that people can shortlist the area of their interests based on the venues/amenities around each neighborhood.

### Exploratory Data Analysis 
Descriptive statistics of the dat

In [ ]:
London_crime.describe()

          Burglary	 CriminalDamage   Drugs	   Other Notifiable   Robbery
                                                    Offences      
count	 33.000000	 33.00000	   33.000000	33.000000	   33.000000	
mean	2069.242424	1941.545455	  1179.212121	479.060606	   682.666667	
std	    737.448644	625.207070	  586.406416	223.298698	  441.425366	
min	    2.000000	2.000000	  10.000000	    6.000000	  4.000000	
25%	    1531.000000	1650.000000	  743.000000	378.000000	  377.000000	
50%	   2071.000000	1989.000000	 1063.000000	490.000000	  599.000000	
75%	   2631.000000	2351.000000	 1617.000000	551.000000	  936.000000	
max	   3402.000000	3219.000000	 2738.000000	1305.000000	  1822.000000



In [ ]:
# use the inline backend to generate the plots within the browser
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Matplotlib version:  2.1.2

#### Check if the column names are strings

In [ ]:
Ld_crime.columns = list(map(str, Ld_crime.columns))

# let's check the column labels types now
all(isinstance(column, str) for column in Ld_crime.columns)

True

###### Sort the total crimes in descenting order to see 5 boroughs with the highest number of crimes

In [ ]:
Ld_crime.sort_values(['Total'], ascending = False, axis = 0, inplace = True )

df_top5 = Ld_crime.head() 
df_top5

Borough    Inner   Status   Local       Political   Headquarters  population
                            authority     control  
    
Barking    NaN     NaN     council      Labour       town sqaure    6545686
and Dagam
    
Barnet     NaN     NaN      council    Conservative   North london   765676
    
Bexley     NaN     NaN      council     Labour       walting street  465465
    
Brent      NaN     NaN      council    Conservative   Brent center    56546
    
Bromley    NaN     NaN      council      Labour         stockwell    345345

### Visualize the five boroughs with the highest number of crimes

In [ ]:
df_tt = df_top5[['Borough','Total']]

df_tt.set_index('Borough',inplace = True)

ax = df_tt.plot(kind='bar', figsize=(10, 6), rot=0)

ax.set_ylabel('Number of Crimes') # add to x-label to the plot
ax.set_xlabel('Borough') # add y-label to the plot
ax.set_title('London Boroughs with the Highest no. of crime') # add title to the plot

# Creating a function to display the percentage.

for p in ax.patches:
    ax.annotate(np.round(p.get_height(),decimals=2), 
                (p.get_x()+p.get_width()/2., p.get_height()), 
                ha='center', 
                va='center', 
                xytext=(0, 10), 
                textcoords='offset points',
                fontsize = 14
               )

plt.show()

##### We'll stay clear from these places :)

##### Sort the total crimes in ascending order to see 5 boroughs with the highest number of crimes

In [ ]:
Ld_crime.sort_values(['Total'], ascending = True, axis = 0, inplace = True )

df_bot5 = Ld_crime.head() 
df_bot5

#### Visualize the five boroughs with the least number of crime

In [ ]:
df_bt = df_bot5[['Borough','Total']]

df_bt.set_index('Borough',inplace = True)

ax = df_bt.plot(kind='bar', figsize=(10, 6), rot=0)

ax.set_ylabel('Number of Crimes') # add to x-label to the plot
ax.set_xlabel('Borough') # add y-label to the plot
ax.set_title('London Boroughs with the least no. of crime') # add title to the plot

# Creating a function to display the percentage.

for p in ax.patches:
    ax.annotate(np.round(p.get_height(),decimals=2), 
                (p.get_x()+p.get_width()/2., p.get_height()), 
                ha='center', 
                va='center', 
                xytext=(0, 10), 
                textcoords='offset points',
                fontsize = 14
               )

plt.show()

##### We'll stay clear from these places :

##### Sort the total crimes in ascending order to see 5 boroughs with the highest number of crime

In [ ]:
Ld_crime.sort_values(['Total'], ascending = True, axis = 0, inplace = True )

df_bot5 = Ld_crime.head() 
df_bot5

Borough    Inner   Status   Local       Political   Headquarters  population
                            authority     control  
    
Barking    NaN     NaN     council      Labour       town sqaure    6545686
and Dagam
    
Barnet     NaN     NaN      council    Conservative   North london   765676
    
Bexley     NaN     NaN      council     Labour       walting street  465465
    
Brent      NaN     NaN      council    Conservative   Brent center    56546
    
Bromley    NaN     NaN      council      Labour         stockwell    345345

### Visualize the five boroughs with the least number of crimes

In [ ]:
df_bt = df_bot5[['Borough','Total']]

df_bt.set_index('Borough',inplace = True)

ax = df_bt.plot(kind='bar', figsize=(10, 6), rot=0)

ax.set_ylabel('Number of Crimes') # add to x-label to the plot
ax.set_xlabel('Borough') # add y-label to the plot
ax.set_title('London Boroughs with the least no. of crime') # add title to the plot

# Creating a function to display the percentage.

for p in ax.patches:
    ax.annotate(np.round(p.get_height(),decimals=2), 
                (p.get_x()+p.get_width()/2., p.get_height()), 
                ha='center', 
                va='center', 
                xytext=(0, 10), 
                textcoords='offset points',
                fontsize = 14
               )

plt.show()

## Visualizing different types of crimes in the borough 'Kingston upon Thames

In [ ]:
df_bc1 =  df_bot5[df_bot5['Borough'] == 'Kingston upon Thames']

df_bc = df_bc1[['Borough','Burglary','Criminal Damage','Drugs','Other Notifiable Offences',
                 'Robbery','Theft and Handling','Violence Against the Person']]


df_bc.set_index('Borough',inplace = True)

ax = df_bc.plot(kind='bar', figsize=(10, 6), rot=0)

ax.set_ylabel('Number of Crimes') # add to x-label to the plot
ax.set_xlabel('Borough') # add y-label to the plot
ax.set_title('London Boroughs with the least no. of crime') # add title to the plot

# Creating a function to display the percentage.

for p in ax.patches:
    ax.annotate(np.round(p.get_height(),decimals=2), 
                (p.get_x()+p.get_width()/2., p.get_height()), 
                ha='center', 
                va='center', 
                xytext=(0, 10), 
                textcoords='offset points',
                fontsize = 14
               )

plt.show()

[51.3937811, 51.41749865, 51.358336, 51.4194499, 51.3678984, 51.4096275, 51.43185, 51.3410523, 51.3909852, 51.4053347, 51.4099994, 51.382484, 51.3926421, 51.3937557, 51.3788758] [-0.2848024, -0.305552805049262, -0.2986216, -0.2653985, -0.3071453, -0.3062621, -0.2581379, -0.3190757, -0.2488979, -0.2634066, -0.2873963, -0.2590897, -0.3143662, -0.3033105, -0.2828604]

In [ ]:
df_neigh = {'Neighborhood': Neighborhood,'Borough':Borough,'Latitude': Latitude,'Longitude':Longitude}
kut_neig = pd.DataFrame(data=df_neigh, columns=['Neighborhood', 'Borough', 'Latitude', 'Longitude'], index=None)

kut_neig

	Neighborhood	      Borough	        Latitude	  Longitude
0	Berrylands	   Kingston upon Thames	   51.393781	 -0.284802
1	Canbury	       Kingston upon Thames	   51.417499	 -0.305553
2	Chessington	   Kingston upon Thames	   51.358336	 -0.298622
3	Coombe	       Kingston upon Thames	   51.419450	 -0.265398
4	Hook	       Kingston upon Thames	   51.367898	-0.307145
5	Kingston       Kingston upon Thames	   51.409627	-0.306262
6	Kingston Vale  Kingston upon Thames	   51.431850	-0.258138
7	Malden Rushett Kingston upon Thames	   51.341052	-0.319076
8	Motspur Park   Kingston upon Thames	   51.390985	-0.248898
9	New Malden	   Kingston upon Thames	   51.405335	-0.263407
10	Norbiton	   Kingston upon Thames	   51.409999	-0.287396
11	Old Malden	   Kingston upon Thames	   51.382484	-0.259090
12	Seething Wells Kingston upon Thames	   51.392642	-0.314366
13	Surbiton	   Kingston upon Thames	   51.393756	-0.303310
14	Tolworth	   Kingston upon Thames	   51.378876	-0.282860

### Get the co-ordinates of Berrylands, London, United Kingdom (The center neighborhood of Kingston upon Thames

In [ ]:
address = 'Berrylands, London, United Kingdom'

geolocator = Nominatim(user_agent="ld_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Berrylands, London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.3937811, -0.2848024.

### The geograpical coordinate of London are 51.3937811, -0.2848024.

In [ ]:
# create map of New York using latitude and longitude values
map_lon = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(kut_neig['Latitude'], kut_neig['Longitude'], kut_neig['Borough'], kut_neig['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lon)  
    
map_lon

# Modelling 
Finding all the venues within a 500 meter radius of each neighborhood.
Perform one hot ecoding on the venues data.
Grouping the venues by the neighborhood and calculating their mean.
Performing a K-means clustering (Defining K = 5)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
kut_venues = getNearbyVenues(names=kut_neig['Neighborhood'],
                                   latitudes=kut_neig['Latitude'],
                                   longitudes=kut_neig['Longitude']
                                  )

Berrylands
Canbury
Chessington
Coombe
Hook
Kingston upon Thames
Kingston Vale
Malden Rushett
Motspur Park
New Malden
Norbiton
Old Malden
Seething Wells
Surbiton
Tolworth

# One hot encoding
URL: https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f

In [ ]:
kut_onehot = pd.get_dummies(kut_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kut_onehot['Neighborhood'] = kut_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kut_onehot.columns[-1]] + list(kut_onehot.columns[:-1])
kut_onehot = kut_onehot[fixed_columns]

kut_onehot.head()

   Neighborhood	    Asian     Auto    Bakery    Bar  Betting   Bistro  Bowling
                 Restaurant	 Garage		     	      Shop	 	        Alley	
0	Berrylands	    0	       0	    0	     0	   0	    0	     0	
1	Berrylands	   	0          0        0        0     0        0        0
2	Berrylands	    0          0        0        0     0        0        0
3	Berrylands	   	0          0        0        0     0        0        0
4	Canbury	        0          0        0        0     0        0        0

# Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
kut_grouped = kut_onehot.groupby('Neighborhood').mean().reset_index()
kut_grouped

Neighborhood	    Asian     Auto    Bakery    Bar  Betting   Bistro  Bowling
                 Restaurant	 Garage		     	      Shop	 	        Alley	
0	Berrylands	    0	       0	    0	     0	   0	    0	     0	
1	hook    	   	0          0        0        0     0        0        0
2	Malden	        0          0        0        0     0        0        0
3	norbiton	   	0          0        0        0     0        0        0
4	Canbury	        0          0        0        0     0        0        0
5   Motspour        0          0        0        0     0        0        0
6   Malden	        0          0        0        0     0        0        0
8	norbiton	   	0          0        0        0     0        0        0
9	Canbury	        0          0        0        0     0        0        0

In [ ]:
kut_grouped.shape

(14, 66)

In [ ]:
num_top_venues = 5

for hood in kut_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = kut_grouped[kut_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
----Berrylands----
                  venue  freq
0              Bus Stop  0.25
1  Gym / Fitness Center  0.25
2                  Park  0.25
3                  Café  0.25
4                   Pub  0.00


----Canbury----
               venue  freq
0                Pub  0.31
1               Park  0.08
2              Hotel  0.08
3  Indian Restaurant  0.08
4  Fish & Chips Shop  0.08


----Chessington----
                   venue  freq
0   Fast Food Restaurant   1.0
1       Asian Restaurant   0.0
2  Portuguese Restaurant   0.0
3         Hardware Store   0.0
4                  Hotel   0.0


----Hook----
               venue  freq
0             Bakery  0.25
1  Indian Restaurant  0.25
2  Fish & Chips Shop  0.25
3  Convenience Store  0.25
4   Asian Restaurant  0.00


----Kingston Vale----
                venue  freq
0       Grocery Store  0.25
1        Soccer Field  0.25
2                 Bar  0.25
3  Italian Restaurant  0.25
4            Platform  0.00


----Kingston upon Thames----
              venue  freq
0              Café  0.13
1       Coffee Shop  0.13
2  Sushi Restaurant  0.07
3      Burger Joint  0.07
4               Pub  0.07


----Malden Rushett----
               venue  freq
0                Pub  0.25
1         Restaurant  0.25
2  Convenience Store  0.25
3      Garden Center  0.25
4               Park  0.00


----Motspur Park----
          venue  freq
0      Bus Stop   0.2
1           Gym   0.2
2    Restaurant   0.2
3          Park   0.2
4  Soccer Field   0.2


----New Malden----
               venue  freq
0                Gym  0.17
1  Indian Restaurant  0.17
2          Gastropub  0.17
3   Sushi Restaurant  0.17
4        Supermarket  0.17


----Norbiton----
                venue  freq
0   Indian Restaurant  0.11
1  Italian Restaurant  0.07
2            Platform  0.07
3                Food  0.07
4                 Pub  0.07


----Old Malden----
           venue  freq
0            Pub  0.33
1  Train Station  0.33
2           Food  0.33
3         Market  0.00
4       Platform  0.00


----Seething Wells----
                venue  freq
0   Indian Restaurant  0.17
1         Coffee Shop  0.13
2  Italian Restaurant  0.09
3                 Pub  0.09
4                Café  0.09


----Surbiton----
            venue  freq
0     Coffee Shop  0.17
1             Pub  0.13
2     Supermarket  0.07
3  Breakfast Spot  0.07
4   Grocery Store  0.03


----Tolworth----
                    venue  freq
0           Grocery Store  0.20
1                Pharmacy  0.13
2                Bus Stop  0.07
3  Furniture / Home Store  0.07
4             Pizza Place  0.07

In [ ]:
kut_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 15 columns):
Neighborhood              15 non-null object
Borough                   15 non-null object
Latitude                  15 non-null float64
Longitude                 15 non-null float64
Cluster Labels            14 non-null float64
1st Most Common Venue     14 non-null object
2nd Most Common Venue     14 non-null object
3rd Most Common Venue     14 non-null object
4th Most Common Venue     14 non-null object
5th Most Common Venue     14 non-null object
6th Most Common Venue     14 non-null object
7th Most Common Venue     14 non-null object
8th Most Common Venue     14 non-null object
9th Most Common Venue     14 non-null object
10th Most Common Venue    14 non-null object
dtypes: float64(3), object(12)
memory usage: 1.8+ KB

#Dropping the row with the NaN value 
kut_merged.dropna(inplace = True)

In [ ]:
kut_merged.shape

(14, 15)

In [ ]:
kut_merged['Cluster Labels'] = kut_merged['Cluster Labels'].astype(int)

In [ ]:
kut_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 14
Data columns (total 15 columns):
Neighborhood              14 non-null object
Borough                   14 non-null object
Latitude                  14 non-null float64
Longitude                 14 non-null float64
Cluster Labels            14 non-null int64
1st Most Common Venue     14 non-null object
2nd Most Common Venue     14 non-null object
3rd Most Common Venue     14 non-null object
4th Most Common Venue     14 non-null object
5th Most Common Venue     14 non-null object
6th Most Common Venue     14 non-null object
7th Most Common Venue     14 non-null object
8th Most Common Venue     14 non-null object
9th Most Common Venue     14 non-null object
10th Most Common Venue    14 non-null object
dtypes: float64(2), int64(1), object(12)
memory usage: 1.8+ KB


#### Visualize the clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kut_merged['Latitude'], kut_merged['Longitude'], kut_merged['Neighborhood'], kut_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters

Each cluster is color coded for the ease of presentation, we can see that majority of the neighborhood falls in the red cluster which is the first cluster. Three neighborhoods have their own cluster (Blue, Purple and Yellow), these are clusters two three and five. The green cluster consists of two neighborhoods which is the 4th cluster.

# Analysis 


Analyse each of the clusters to identify the characteristics of each cluster and the neighborhoods in them.

#### Examine the first cluster

In [ ]:
kut_merged[kut_merged['Cluster Labels'] == 0]

    Neighborhood	Borough	  	Cluster  1st Common 2nd Common  3rd  Common
                                           Venue	  Venue	     Venue
1	Canbury	      Kingston 		0	       Pub	      Café	      Plaza	
4	Hook	      Kingston 		0	      Bakery	Convenience   Store	
5	Kingston 	  Kingston  	0	    CoffeeShop	  Café	    Burger Joint	
7	Malden Rushet Kingston 		0	 ConvenienceStore  Pub	    Garden Center		
9	New Malden	  Kingston 		0	   Gastropub	   Gym	   Sushi Restaurant	
10	Norbiton	  Kingston 		0	 IndianRestaurant	Pub	      Food	
12	SeethingWells Kingston 		0	 IndianRestaurant	Coffee    food Shop		
13	Surbiton	  Kingston 		0	   CoffeeShop	   Pub	     Supermarket		
14	Tolworth	  Kingston 		0	  grocery Store     Market      store

The cluster one is the biggest cluster with 9 of the 15 neighborhoods in the borough Kingston upon Thames. Upon closely examining these neighborhoods we can see that the most common venues in these neighborhoods are Restaurants, Pubs, Cafe, Supermarkets, and stores.

#### Examine the second cluste

In [ ]:
kut_merged[kut_merged['Cluster Labels'] == 2]

  Neighborhood	  Borough	  Cluster  1st Common  2nd Common   3rd  Common
                                           Venue	  Venue	     Venue

13	Surbiton	  Kingston 		0	   CoffeeShop	   Pub	     Supermarket

#### Examine the third cluster

In [ ]:
kut_merged[kut_merged['Cluster Labels'] == 2]

   Neighborhood	   Borough	  Cluster  1st Common  2nd Common   3rd  Common
                                           Venue	  Venue	     Venue
        
7	Malden Rushet Kingston 		0	 ConvenienceStore  Pub	    Garden Center	

#### Examine the forth cluster

In [ ]:
kut_merged[kut_merged['Cluster Labels'] == 3]

   Neighborhood	   Borough	  Cluster  1st Common  2nd Common   3rd  Common
                                           Venue	  Venue	     Venue
        
7	Malden Rushet Kingston 		0	 ConvenienceStore  Pub	    Garden Center		
9	New Malden	  Kingston 		0	   Gastropub	   Gym	   Sushi Restaurant

### Examine the fifth cluster

In [ ]:
kut_merged[kut_merged['Cluster Labels'] == 4]

    Neighborhood	   Borough	  Cluster  1st Common  2nd Common   3rd  Common
                                             Venue	      Venue	     Venue
        
5	Kingston 	  Kingston  	0	    CoffeeShop	    Café	    Burger Joint	

# Results and Discussion 
The aim of this project is to help people who want to relocate to the safest borough in London, expats can chose the neighborhoods to which they want to relocate based on the most common venues in it. For example if a person is looking for a neighborhood with good connectivity and public transportation we can see that Clusters 3 and 4 have Train stations and Bus stops as the most common venues. If a person is looking for a neighborhood with stores and restaurants in a close proximity then the neighborhoods in the first cluster is suitable. For a family I feel that the neighborhoods in Cluster 4 are more suitable dues to the common venues in that cluster, these neighborhoods have common venues such as Parks, Gym/Fitness centers, Bus Stops, Restaurants, Electronics Stores and Soccer fields which is ideal for a family.

# Conclusion 
This project helps a person get a better understanding of the neighborhoods with respect to the most common venues in that neighborhood. It is always helpful to make use of technology to stay one step ahead i.e. finding out more about places before moving into a neighborhood. We have just taken safety as a primary concern to shortlist the borough of London. The future of this project includes taking other factors such as cost of living in the areas into consideration to shortlist the borough based on safety and a predefined budget.